In [4]:
import math
import logging
import random
import networkx as nx
import numpy as np
from itertools import combinations


In [ ]:
class Problem:
    _graph: nx.Graph
    _alpha: float
    _beta: float

    def __init__(
        self,
        num_cities: int = 30,
        *,
        alpha: float = 1.0,
        beta: float = 1.0,
        density: float = 0.5,
        seed: int = 42,
    ):
        rng = np.random.default_rng(seed)
        self._alpha = alpha
        self._beta = beta
        cities = rng.random(size=(num_cities, 2))
        cities[0, 0] = cities[0, 1] = 0.5

        self._graph = nx.Graph()
        self._graph.add_node(0, pos=(cities[0, 0], cities[0, 1]), gold=0)
        for c in range(1, num_cities):
            self._graph.add_node(c, pos=(cities[c, 0], cities[c, 1]), gold=(1 + 999 * rng.random()))

        tmp = cities[:, np.newaxis, :] - cities[np.newaxis, :, :]
        d = np.sqrt(np.sum(np.square(tmp), axis=-1))
        for c1, c2 in combinations(range(num_cities), 2):
            if rng.random() < density or c2 == c1 + 1:
                self._graph.add_edge(c1, c2, dist=d[c1, c2])
        if not nx.is_connected(self._graph):
             components = list(nx.connected_components(self._graph))
             for i in range(len(components)-1):
                 u = list(components[i])[0]
                 v = list(components[i+1])[0]
                 self._graph.add_edge(u, v, dist=np.linalg.norm(cities[u]-cities[v]))
        self._shortest_paths_lens = dict(nx.all_pairs_dijkstra_path_length(self._graph, weight='dist'))
        self._paths = dict(nx.all_pairs_dijkstra_path(self._graph, weight='dist'))
        
        self._edge_dists = {}
        for u, v, data in self._graph.edges(data=True):
            self._edge_dists[(u, v)] = data['dist']
            self._edge_dists[(v, u)] = data['dist']

    @property
    def graph(self) -> nx.Graph: return nx.Graph(self._graph)
    @property
    def alpha(self): return self._alpha
    @property
    def beta(self): return self._beta

    def cost(self, path, weight):
        dist = nx.path_weight(self._graph, path, weight='dist')
        return dist + (self._alpha * dist * weight) ** self._beta

    def _get_real_path_cost(self, u, v, load):
        if u == v: return 0.0
        
        path = self._paths[u][v]
        total_cost = 0.0
        
        for i in range(len(path) - 1):
            n1, n2 = path[i], path[i+1]
            d = self._edge_dists[(n1, n2)]
            segment_cost = d + (self._alpha * d * load) ** self._beta
            total_cost += segment_cost
            
        return total_cost

    def baseline(self):
        total_cost = 0
        for dest, path in nx.single_source_dijkstra_path(
            self._graph, source=0, weight='dist'
        ).items():
            if dest == 0: continue
            cost = 0
            for c1, c2 in zip(path, path[1:]):
                cost += self.cost([c1, c2], 0)
                cost += self.cost([c1, c2], self._graph.nodes[dest]['gold'])
            total_cost += cost
        return total_cost

    def split(self, sequence):
        n = len(sequence)
        golds = [self._graph.nodes[node]['gold'] for node in sequence]
        
        V = [float('inf')] * (n + 1); V[0] = 0.0
        P = [0] * (n + 1)

        for i in range(n):
            if V[i] == float('inf'): continue
            
            current_load = 0.0
            trip_cost = 0.0
            u = 0 
            
            for j in range(i + 1, n + 1):
                v = sequence[j-1]
                gold_v = golds[j-1]
                
                try: cost_uv = self._get_real_path_cost(u, v, current_load)
                except KeyError: break 
                trip_cost += cost_uv
                
                current_load += gold_v
                
                try: return_cost = self._get_real_path_cost(v, 0, current_load)
                except KeyError: break
                    
                total_trip_cost = trip_cost + return_cost
                
                if V[i] + total_trip_cost < V[j]:
                    V[j] = V[i] + total_trip_cost
                    P[j] = i
                
                u = v

        routes = []
        curr = n
        if V[n] == float('inf'): return float('inf'), []

        while curr > 0:
            start = P[curr]
            routes.append([0] + sequence[start:curr] + [0])
            curr = start
        routes.reverse()
        return V[n], routes
    
    def generate_nearest_neighbor_individual(self):
        unvisited = set(range(1, len(self._graph)))
        curr = 0; tour = []
        while unvisited:
            next_n = min(unvisited, key=lambda n: self._shortest_paths_lens[curr][n])
            tour.append(next_n); unvisited.remove(next_n); curr = next_n
        return tour

    def generate_cost_aware_greedy(self):
        unvisited = set(range(1, len(self._graph)))
        curr = 0; load = 0; tour = []
        while unvisited:
            best_n, best_inc = None, float('inf')
            for cand in unvisited:
                d = self._shortest_paths_lens[curr][cand]
                step = d + (self._alpha * d * load) ** self._beta
                if step < best_inc:
                    best_inc = step; best_n = cand
            tour.append(best_n); unvisited.remove(best_n); curr = best_n
            load += self._graph.nodes[best_n]['gold']
        return tour
    
    def generate_grasp_individual(self, k=3):
        unvisited = list(range(1, len(self._graph))); curr = 0; tour = []
        while unvisited:
            unvisited.sort(key=lambda n: self._shortest_paths_lens[curr][n])
            cand = unvisited[:k]; nxt = random.choice(cand)
            tour.append(nxt); unvisited.remove(nxt); curr = nxt
        return tour

    def generate_clarke_wright_individual(self):
        n = len(self._graph) - 1; nodes = list(range(1, n + 1))
        routes = {i: [i] for i in nodes}; node_to_route = {i: i for i in nodes}
        savings = []; golds = {i: self._graph.nodes[i]['gold'] for i in nodes}

        for i in range(1, n + 1):
            for j in range(i + 1, n + 1):
                try:
                    d0i = self._shortest_paths_lens[0][i]; di0 = self._shortest_paths_lens[i][0]
                    d0j = self._shortest_paths_lens[0][j]; dj0 = self._shortest_paths_lens[j][0]
                    dij = self._shortest_paths_lens[i][j]; dji = self._shortest_paths_lens[j][i]
                except KeyError: continue
                
                gi = golds[i]; gj = golds[j]
                
                base = (d0i + (di0 + (self._alpha * di0 * gi)**self._beta)) + \
                       (d0j + (dj0 + (self._alpha * dj0 * gj)**self._beta))
                
                cij = d0i + (dij + (self._alpha * dij * gi)**self._beta) + \
                      (dj0 + (self._alpha * dj0 * (gi + gj))**self._beta)
                cji = d0j + (dji + (self._alpha * dji * gj)**self._beta) + \
                      (di0 + (self._alpha * di0 * (gj + gi))**self._beta)
                
                sav = base - min(cij, cji)
                if sav > 0: savings.append((sav, i, j))
        
        savings.sort(key=lambda x: x[0], reverse=True)
        
        for _, i, j in savings:
            ri = node_to_route[i]; rj = node_to_route[j]
            if ri != rj:
                route_i = routes[ri]; route_j = routes[rj]
                if route_i[-1] == i and route_j[0] == j: new = route_i + route_j
                elif route_j[-1] == j and route_i[0] == i: new = route_j + route_i
                elif route_i[-1] == i and route_j[-1] == j: new = route_i + route_j[::-1]
                elif route_i[0] == i and route_j[0] == j: new = route_i[::-1] + route_j
                else: continue
                del routes[ri]; del routes[rj]
                routes[new[0]] = new
                for node in new: node_to_route[node] = new[0]
        return [n for r in routes.values() for n in r]

    def _broken_pairs_distance(self, chrom1, chrom2):
        n = len(chrom1)
        edges2 = set()
        for i in range(n - 1):
            u, v = chrom2[i], chrom2[i+1]
            edges2.add((min(u, v), max(u, v))) 
        
        broken = 0
        for i in range(n - 1):
            u, v = chrom1[i], chrom1[i+1]
            if (min(u, v), max(u, v)) not in edges2:
                broken += 1
        return broken / (n - 1)

    def _update_biased_fitness(self, population):
        pop_size = len(population)
        sample_size = pop_size 
        
        distances = np.zeros((sample_size, sample_size))
        
        for i in range(sample_size):
            for j in range(i + 1, sample_size):
                d = self._broken_pairs_distance(population[i][1], population[j][1])
                distances[i][j] = distances[j][i] = d
        
        div_scores = []
        k_neigh = max(1, int(sample_size / 2))
        for i in range(sample_size):
            dists = sorted(distances[i])
            avg = sum(dists[1:k_neigh+1]) / k_neigh
            div_scores.append((avg, i))
        div_scores.sort(key=lambda x: x[0], reverse=True)
        
        div_ranks = {orig: r+1 for r, (_, orig) in enumerate(div_scores)}
        
        cost_scores = [(population[i][0], i) for i in range(sample_size)]
        cost_scores.sort(key=lambda x: x[0])
        cost_ranks = {orig: r+1 for r, (_, orig) in enumerate(cost_scores)}
        
        elite_ratio = 0.9 
        
        fitness_list = []
        for i in range(sample_size):
            r_c = cost_ranks[i]
            r_d = div_ranks[i] 
            biased_fit = r_c + (1.0 - elite_ratio) * r_d
            
            fitness_list.append([biased_fit, population[i][0], population[i][1], population[i][2]])
            
        best_idx = [i for i, r in cost_ranks.items() if r == 1][0]
        unique_idx = [i for i, r in div_ranks.items() if r == 1][0]
        
        print(f"DEBUG:")
        print(f"  > BEST COST (Idx {best_idx}): Cost={population[best_idx][0]:.1f} | RankC={cost_ranks[best_idx]} | RankD={div_ranks[best_idx]} => Fit={fitness_list[best_idx][0]:.2f}")
        print(f"  > MOST UNIQUE (Idx {unique_idx}): Cost={population[unique_idx][0]:.1f} | RankC={cost_ranks[unique_idx]} | RankD={div_ranks[unique_idx]} => Fit={fitness_list[unique_idx][0]:.2f}")
        print("-" * 50)
            
        return fitness_list

    def _smart_tournament_selection(self, population_with_fitness, k=2):
        """
        Torneo basato sulla Biased Fitness.
        Ritorna l'intero pacchetto [Fitness, Costo, Cromosoma, Rotte] 
        per permettere al test di analizzare i costi.
        """
        candidates = random.sample(population_with_fitness, k)
        best = min(candidates, key=lambda x: x[0]) 
        return best 
    
    def _ordered_crossover(self, parent1, parent2):
    
        size = len(parent1)
        a, b = sorted(random.sample(range(size), 2))
        
        child = [-1] * size
        child[a:b+1] = parent1[a:b+1]
        current_genes = set(child[a:b+1])
        
        p2_vals = [x for x in parent2 if x not in current_genes]
        
        cur = 0
        for i in range(size):
            if child[i] == -1:
                child[i] = p2_vals[cur]
                cur += 1
                
        return child
    
    def _ls_relocate(self, tour):
 
        n = len(tour)
        i = random.randint(0, n - 1)
        node = tour[i]
        
        j = random.randint(0, n - 2) 
        
        new_tour = tour[:i] + tour[i+1:] 
        new_tour.insert(j, node)         
        return new_tour

    def _ls_swap(self, tour):
        n = len(tour)
        i, j = random.sample(range(n), 2)
        new_tour = tour[:]
        new_tour[i], new_tour[j] = new_tour[j], new_tour[i]
        return new_tour

    def _ls_2opt(self, tour):
        n = len(tour)
        i, j = sorted(random.sample(range(n), 2))
        
        new_tour = tour[:]
        new_tour[i:j+1] = new_tour[i:j+1][::-1]
        return new_tour

    def educate(self, chromosome):
        max_attempts = 50
        improved = True
        
        best_chrom = chromosome[:]
        best_cost, _ = self.split(best_chrom) 
        
        while improved:
            improved = False
            
            for _ in range(max_attempts):
                cand = self._ls_relocate(best_chrom)
                c_cand, _ = self.split(cand)
                if c_cand < best_cost:
                    best_cost = c_cand
                    best_chrom = cand
                    improved = True
                    break
            if improved: continue

            for _ in range(max_attempts):
                cand = self._ls_swap(best_chrom)
                c_cand, _ = self.split(cand)
                if c_cand < best_cost:
                    best_cost = c_cand
                    best_chrom = cand
                    improved = True
                    break
            if improved: continue

            for _ in range(max_attempts):
                cand = self._ls_2opt(best_chrom)
                c_cand, _ = self.split(cand)
                if c_cand < best_cost:
                    best_cost = c_cand
                    best_chrom = cand
                    improved = True
                    break
            
        return best_chrom

In [ ]:
def initialization_v2(problem, pop_size):

    population = []
    
    try: population.append(problem.generate_clarke_wright_individual())
    except: pass
    
    population.append(problem.generate_cost_aware_greedy())
    population.append(problem.generate_nearest_neighbor_individual())
    
    k_grasp = 6 if problem.beta >= 1.5 else 3
    grasp_quota = int(pop_size * 0.2)
    for _ in range(grasp_quota):
        population.append(problem.generate_grasp_individual(k=k_grasp))
        
    while len(population) < pop_size:
        l = list(range(1, len(problem._graph)))
        random.shuffle(l)
        population.append(l)
        
    return population




In [ ]:
def run_full_matrix_benchmark(num_cities=100, pop_size=100):
    print(f"{'='*85}")
    print(f"   FULL MATRIX BENCHMARK: INIT V2 ({num_cities} Città)")
    print(f"{'='*85}")
    print(f"{'SCENARIO':<35} | {'BASELINE':<12} | {'MIGLIOR INIT':<12} | {'GAP':<10}")
    print("-" * 85)

    betas = [0.5, 1.0, 2.0]
    densities = [0.2, 0.5, 1.0]

    for beta in betas:
        for dens in densities:
            sc_name = f"Beta={beta} | Dens={dens}"
            if beta < 1: sc_name += " (Economy)"
            elif beta > 1: sc_name += " (Punitivo)"
            
            else: sc_name += " (Standard)"
            
            try:
                p = Problem(num_cities, alpha=1.0, beta=beta, density=dens, seed=42)
                baseline = p.baseline()
                
                pop = initialization_v2(p, pop_size)
                
                best_cost = float('inf')
                for ind in pop:
                    c, _ = p.split(ind)
                    if c < best_cost: best_cost = c
                
                gap = ((baseline - best_cost) / baseline) * 100
                gap_str = f"{gap:+.2f}%"
                
                print(f"{sc_name:<35} | {baseline:<12.2f} | {best_cost:<12.2f} | {gap_str:<10}")
                
            except Exception as e:
                print(f"{sc_name:<35} | ERROR: {e}")

run_full_matrix_benchmark(num_cities=100, pop_size=100)

   FULL MATRIX BENCHMARK: INIT V2 (100 Città)
SCENARIO                            | BASELINE     | MIGLIOR INIT | GAP       
-------------------------------------------------------------------------------------
Beta=0.5 | Dens=0.2 (Economy)       | 2231.05      | 1740.78      | +21.97%   
Beta=0.5 | Dens=0.5 (Economy)       | 1535.71      | 1236.58      | +19.48%   
Beta=0.5 | Dens=1.0 (Economy)       | 1292.63      | 1101.51      | +14.79%   


KeyboardInterrupt: 

In [ ]:
def run_selection_quality_test(num_cities=100, pop_size=100):
    print(f"{'='*80}")
    print(f"   TEST BIASED FITNESS: SELEZIONE GENITORI (Vidal Style)")
    print(f"{'='*80}\n")
    
    scenarios = [
        {"name": "Economy (B=0.5)", "b": 0.5, "d": 0.5},
        {"name": "Standard (B=1.0)", "b": 1.0, "d": 0.5},
        {"name": "Punitivo (B=2.0)", "b": 2.0, "d": 0.5},
        {"name": "Sparso (B=2, D=0.2)","b": 2.0, "d": 0.2},
    ]
    
    for sc in scenarios:
        try:
            p = Problem(num_cities, alpha=1.0, beta=sc['b'], density=sc['d'], seed=42)
            raw_pop = initialization_v2(p, pop_size)
            
            eval_pop = []
            for ind in raw_pop:
                c, _ = p.split(ind)
                eval_pop.append([c, ind, []])
            
            best_cost = min(x[0] for x in eval_pop)
            
            pop_fit = p._update_biased_fitness(eval_pop)
            
            num_trials = 200
            best_selected = 0
            diffs = []
            
            for _ in range(num_trials):
                p1_data = p._smart_tournament_selection(pop_fit, k=2)
                p2_data = p._smart_tournament_selection(pop_fit, k=2)
                
                c1 = p1_data[1]
                c2 = p2_data[1]
                
                if abs(c1 - best_cost) < 1e-4 or abs(c2 - best_cost) < 1e-4:
                    best_selected += 1
                
                gap = abs(c1 - c2) / (min(c1, c2) + 1e-9) * 100
                diffs.append(gap)
            
            sel_rate = best_selected / num_trials * 100
            avg_diff = sum(diffs) / len(diffs)
            
            print(f"---> {sc['name']}")
            print(f"     Best Cost Global: {best_cost:.2f}")
            print(f"     Frequenza Selezione Best: {sel_rate:.1f}%")
            print(f"     Differenza Costo Media Genitori: {avg_diff:.2f}%")
            
            if sel_rate > 90: print("     [!] WARNING: Dominanza eccessiva del migliore.")
            elif avg_diff < 1.0: print("     [!] WARNING: Genitori troppo simili (scarsa diversità).")
            else: print("     [OK] Selezione bilanciata.")
            print("")
            
        except Exception as e: 
            print(f"Error in {sc['name']}: {e}")

run_selection_quality_test()

   TEST BIASED FITNESS: SELEZIONE GENITORI (Vidal Style)

DEBUG:
  > BEST COST (Idx 0): Cost=1236.6 | RankC=1 | RankD=98 => Fit=10.80
  > MOST UNIQUE (Idx 70): Cost=1521.6 | RankC=98 | RankD=1 => Fit=98.10
--------------------------------------------------
---> Economy (B=0.5)
     Best Cost Global: 1236.58
     Frequenza Selezione Best: 4.0%
     Differenza Costo Media Genitori: 6.88%
     [OK] Selezione bilanciata.

DEBUG:
  > BEST COST (Idx 0): Cost=18800.7 | RankC=1 | RankD=78 => Fit=8.80
  > MOST UNIQUE (Idx 76): Cost=18811.3 | RankC=89 | RankD=1 => Fit=89.10
--------------------------------------------------
---> Standard (B=1.0)
     Best Cost Global: 18800.71
     Frequenza Selezione Best: 2.0%
     Differenza Costo Media Genitori: 0.01%
     [!] WARNING: Genitori troppo simili (scarsa diversità).

DEBUG:
  > BEST COST (Idx 0): Cost=3888666.0 | RankC=1 | RankD=77 => Fit=8.70
  > MOST UNIQUE (Idx 54): Cost=4599913.5 | RankC=67 | RankD=1 => Fit=67.10
-----------------------------

In [ ]:
def run_crossover_improvement_test(num_cities=50, pop_size=50):
    print(f"{'='*80}")
    print(f"   TEST CROSSOVER IMPROVEMENT: FIGLI vs POPOLAZIONE")
    print(f"{'='*80}\n")
    
    scenarios = [
        {"name": "Economy (B=0.5)", "b": 0.5, "d": 0.5},
        {"name": "Standard (B=1.0)", "b": 1.0, "d": 0.5},
        {"name": "Punitivo (B=2.0)", "b": 2.0, "d": 0.5},
        {"name": "Sparso (B=2, D=0.2)","b": 2.0, "d": 0.2},
    ]
    
    for sc in scenarios:
        try:
            p = Problem(num_cities, alpha=1.0, beta=sc['b'], density=sc['d'], seed=42)
            raw_pop = initialization_v2(p, pop_size)
            
            eval_pop = [] 
            costs = []
            for ind in raw_pop:
                c, r = p.split(ind)
                eval_pop.append([c, ind, r])
                costs.append(c)
                
            best_cost = min(costs)
            avg_cost = sum(costs) / len(costs)
            pop_with_fit = p._update_biased_fitness(eval_pop)
            
            num_trials = 200
            better_than_avg = 0
            better_than_best = 0
            
            for _ in range(num_trials):
                p1_data = p._smart_tournament_selection(pop_with_fit, k=2)
                p2_data = p._smart_tournament_selection(pop_with_fit, k=2)
                
                child_chrom = p._ordered_crossover(p1_data[2], p2_data[2])
                
                child_cost, _ = p.split(child_chrom)
                
                if child_cost < avg_cost:
                    better_than_avg += 1
                if child_cost < best_cost:
                    better_than_best += 1
                    
            pct_avg = (better_than_avg / num_trials) * 100
            pct_best = (better_than_best / num_trials) * 100
            
            print(f"---> {sc['name']}")
            print(f"     Pop Best: {best_cost:.2f} | Pop Avg: {avg_cost:.2f}")
            print(f"     Figli > Media Pop: {pct_avg:.1f}%")
            print(f"     Figli > Best Pop:  {pct_best:.1f}%")
            
            if pct_avg < 10:
                print("     [!] WARNING: Il Crossover da solo fatica a produrre soluzioni decenti.")
                print("         (Normale in VRP: conferma la necessità della Local Search post-crossover)")
            else:
                print("     [OK] Il Crossover preserva buoni tratti.")
            print("")
            
        except Exception as e:
            print(f"Error in {sc['name']}: {e}")

run_crossover_improvement_test(num_cities=50, pop_size=50)

   TEST CROSSOVER IMPROVEMENT: FIGLI vs POPOLAZIONE

DEBUG:
  > BEST COST (Idx 0): Cost=656.1 | RankC=1 | RankD=48 => Fit=5.80
  > MOST UNIQUE (Idx 43): Cost=789.4 | RankC=21 | RankD=1 => Fit=21.10
--------------------------------------------------
---> Economy (B=0.5)
     Pop Best: 656.14 | Pop Avg: 780.15
     Figli > Media Pop: 57.5%
     Figli > Best Pop:  0.0%
     [OK] Il Crossover preserva buoni tratti.

DEBUG:
  > BEST COST (Idx 0): Cost=9400.5 | RankC=1 | RankD=38 => Fit=4.80
  > MOST UNIQUE (Idx 14): Cost=9408.3 | RankC=27 | RankD=1 => Fit=27.10
--------------------------------------------------
---> Standard (B=1.0)
     Pop Best: 9400.54 | Pop Avg: 9407.69
     Figli > Media Pop: 60.0%
     Figli > Best Pop:  0.0%
     [OK] Il Crossover preserva buoni tratti.

DEBUG:
  > BEST COST (Idx 9): Cost=1805862.6 | RankC=1 | RankD=45 => Fit=5.50
  > MOST UNIQUE (Idx 23): Cost=1876144.0 | RankC=45 | RankD=1 => Fit=45.10
--------------------------------------------------
---> Punitiv

In [ ]:
def run_education_impact_test(num_cities=50, pop_size=50):
    print(f"\n{'='*80}")
    print(f"   TEST EDUCATION IMPACT: DAL CROSSOVER ALL'OTTIMO LOCALE")
    print(f"{'='*80}\n")
    
    scenarios = [
        {"name": "Economy (B=0.5)", "b": 0.5, "d": 0.5},
        {"name": "Standard (B=1.0)", "b": 1.0, "d": 0.5},
        {"name": "Punitivo (B=2.0)", "b": 2.0, "d": 0.5},
        {"name": "Sparso (B=2, D=0.2)","b": 2.0, "d": 0.2},
    ]
    
    for sc in scenarios:
        try:
            p = Problem(num_cities, alpha=1.0, beta=sc['b'], density=sc['d'], seed=42)
            raw_pop = initialization_v2(p, pop_size)
            
            eval_pop = []
            costs = []
            for ind in raw_pop:
                c, r = p.split(ind)
                eval_pop.append([c, ind, r])
                costs.append(c)
            
            pop_best_cost = min(costs)
            pop_with_fit = p._update_biased_fitness(eval_pop)
            
            num_trials = 100
            edu_wins = 0      
            beats_best = 0 
            avg_improv = [] 
            
            for _ in range(num_trials):
                p1 = p._smart_tournament_selection(pop_with_fit, k=2)
                p2 = p._smart_tournament_selection(pop_with_fit, k=2)
                child_raw = p._ordered_crossover(p1[2], p2[2])
                
                cost_raw, _ = p.split(child_raw)
                
                child_edu = p.educate(child_raw)
                cost_edu, _ = p.split(child_edu)
                
                if cost_edu < cost_raw:
                    edu_wins += 1
                    improv = (cost_raw - cost_edu) / cost_raw * 100
                    avg_improv.append(improv)
                else:
                    avg_improv.append(0.0)
                    
                if cost_edu < pop_best_cost: 
                    beats_best += 1
            
            avg_imp_val = sum(avg_improv)/len(avg_improv)
            beats_best_pct = (beats_best / num_trials) * 100
            
            print(f"---> {sc['name']}")
            print(f"     Pop Best Cost: {pop_best_cost:.2f}")
            print(f"     Education migliora Crossover: {edu_wins}/{num_trials} volte")
            print(f"     Miglioramento Medio (Gap):    {avg_imp_val:.2f}%")
            print(f"     FIGLI CHE BATTONO IL BEST:    {beats_best_pct:.1f}%  <-- IL DATO CHIAVE")
            
            if beats_best_pct > 0:
                print("     [SUCCESS] L'Education permette di superare i genitori!")
            else:
                print("     [WAIT] Ancora difficile battere il C&W (normale in Gen 0).")
            print("")
            
        except Exception as e:
            print(f"Error in {sc['name']}: {e}")

run_education_impact_test(num_cities=50, pop_size=50)


   TEST EDUCATION IMPACT: DAL CROSSOVER ALL'OTTIMO LOCALE

DEBUG:
  > BEST COST (Idx 0): Cost=656.1 | RankC=1 | RankD=48 => Fit=5.80
  > MOST UNIQUE (Idx 23): Cost=803.5 | RankC=40 | RankD=1 => Fit=40.10
--------------------------------------------------
---> Economy (B=0.5)
     Pop Best Cost: 656.14
     Education migliora Crossover: 100/100 volte
     Miglioramento Medio (Gap):    11.13%
     FIGLI CHE BATTONO IL BEST:    4.0%  <-- IL DATO CHIAVE
     [SUCCESS] L'Education permette di superare i genitori!

DEBUG:
  > BEST COST (Idx 0): Cost=9400.5 | RankC=1 | RankD=39 => Fit=4.90
  > MOST UNIQUE (Idx 25): Cost=9408.6 | RankC=40 | RankD=1 => Fit=40.10
--------------------------------------------------
---> Standard (B=1.0)
     Pop Best Cost: 9400.54
     Education migliora Crossover: 100/100 volte
     Miglioramento Medio (Gap):    0.05%
     FIGLI CHE BATTONO IL BEST:    8.0%  <-- IL DATO CHIAVE
     [SUCCESS] L'Education permette di superare i genitori!

DEBUG:
  > BEST COST (Idx

In [ ]:
import time

class GeneticSolver:
    def __init__(self, problem, pop_size=50, elite_ratio=0.9):
        self.p = problem
        self.pop_size = pop_size
        self.elite_ratio = elite_ratio 
        self.population = []
        self.best_solution = None
        self.best_cost = float('inf')
        self.history = []

    def run(self, max_generations=1000, time_limit=600):

        start_time = time.time()
        print(f"--- AVVIO HGS (Pop={self.pop_size}, MaxGen={max_generations}) ---")
        
        print("1. Generazione Popolazione Iniziale...")
        raw_pop = initialization_v2(self.p, self.pop_size)
        
        self.population = []
        for chrom in raw_pop:
            c, r = self.p.split(chrom)
            self.population.append([c, chrom, r])
            if c < self.best_cost:
                self.best_cost = c
                self.best_solution = r
    
        self.population.sort(key=lambda x: x[0])
        print(f"   Best Iniziale: {self.best_cost:.2f}")
        
        gen = 0
        stagnation = 0
        
        while gen < max_generations:
            if time.time() - start_time > time_limit:
                print("!! Time Limit Raggiunto !!")
                break
                
            pop_with_fit = self.p._update_biased_fitness(self.population)
            
            p1 = self.p._smart_tournament_selection(pop_with_fit, k=2)
            p2 = self.p._smart_tournament_selection(pop_with_fit, k=2)
            
            child_chrom_raw = self.p._ordered_crossover(p1[2], p2[2])
            
            child_chrom_edu = self.p.educate(child_chrom_raw)
            child_cost, child_routes = self.p.split(child_chrom_edu)
        
            is_clone = False
            for ind in self.population:
                if abs(ind[0] - child_cost) < 1e-4: 
                    if self.p._broken_pairs_distance(ind[1], child_chrom_edu) == 0:
                        is_clone = True
                        break
            
            if not is_clone:
                self.population.append([child_cost, child_chrom_edu, child_routes])
                
                if child_cost < self.best_cost - 1e-4:
                    gap = ((self.best_cost - child_cost) / self.best_cost) * 100
                    print(f"   ★ [Gen {gen}] NEW BEST: {child_cost:.2f} (Migliorato di -{gap:.2f}%)")
                    self.best_cost = child_cost
                    self.best_solution = child_routes
                    stagnation = 0
                else:
                    stagnation += 1
                
                if len(self.population) > self.pop_size:
                    temp_pop_fit = self.p._update_biased_fitness(self.population)
                    
                    worst_ind = max(temp_pop_fit, key=lambda x: x[0])
                    
                    for i, ind in enumerate(self.population):
                        if ind[1] == worst_ind[2]: 
                            self.population.pop(i)
                            break
            else:
                pass
        
            gen += 1
            if gen % 50 == 0:
                print(f"   Gen {gen:<4} | Best: {self.best_cost:<10.2f} | Pop Avg: {sum(x[0] for x in self.population)/len(self.population):.2f}")
        total_time = time.time() - start_time
        print(f"--- FINE HGS ---")
        print(f"Tempo Totale: {total_time:.2f}s")
        print(f"Miglior Costo Trovato: {self.best_cost:.2f}")
        return self.best_cost, self.best_solution

In [ ]:
p_final = Problem(100, density=0.2, alpha=1, beta=2)
baseline = p_final.baseline()

print(f"=== BENCHMARK FINALE: HGS vs BASELINE ===")
print(f"Scenario: Punitivo (100 Città, Beta=1.0)")
print(f"Baseline (A/R): {baseline:.2f}")

solver = GeneticSolver(p_final, pop_size=40, elite_ratio=0.9) 

final_cost, final_routes = solver.run(max_generations=200, time_limit=150)

print("\n" + "="*50)
print("RISULTATI FINALI")
print("="*50)
print(f"Baseline Cost: {baseline:.2f}")
print(f"HGS Best Cost: {final_cost:.2f}")
gap = ((baseline - final_cost) / baseline) * 100
print(f"Miglioramento: {gap:.2f}%")
print(f"Numero Viaggi: {len(final_routes)}")

=== BENCHMARK FINALE: HGS vs BASELINE ===
Scenario: Punitivo (100 Città, Beta=1.0)
Baseline (A/R): 5334401.93
--- AVVIO HGS (Pop=40, MaxGen=200) ---
1. Generazione Popolazione Iniziale...


NameError: name 'initialization_v2' is not defined

In [ ]:
import time
import random
import numpy as np

def split_heavy(self, seq):
    n = len(seq)
    V = [1e30] * (n + 1); V[0] = 0
    P = [0] * (n + 1)
    
    max_tour_len = 40 
    
    for i in range(n):
        if V[i] >= 1e29: continue
        load = 0.0; cost = 0.0; u = 0
        limit = min(n + 1, i + 1 + max_tour_len)
        
        for j in range(i + 1, limit):
            v = seq[j-1]
            try: cost += self._get_path_cost(u, v, load)
            except: break
            load += self._golds[v]
            try: cost_ret = self._get_path_cost(v, 0, load)
            except: break
            
            total = cost + cost_ret
            if V[i] + total < V[j]:
                V[j] = V[i] + total
                P[j] = i
            u = v
            
    routes = []; curr = n
    if V[n] >= 1e29: return 1e30, []
    while curr > 0:
        prev = P[curr]
        routes.append([0] + seq[prev:curr] + [0])
        curr = prev
    return V[n], routes[::-1]

def educate_heavy(self, chrom):
    best = chrom[:]
    best_c, _ = self.split(best)
    improved = True
    attempts = 500 
    
    while improved:
        improved = False
        for _ in range(attempts):
            n = len(best); i = random.randint(0, n-1); j = random.randint(0, n-2)
            cand = best[:i] + best[i+1:]; cand.insert(j, best[i])
            c, _ = self.split(cand)
            if c < best_c: best_c=c; best=cand; improved=True; break
        if improved: continue
        
        for _ in range(attempts):
            n = len(best); i, j = random.sample(range(n), 2)
            cand = best[:]; cand[i], cand[j] = cand[j], cand[i]
            c, _ = self.split(cand)
            if c < best_c: best_c=c; best=cand; improved=True; break
        if improved: continue

        for _ in range(attempts):
            n = len(best); i, j = sorted(random.sample(range(n), 2))
            cand = best[:]; cand[i:j+1] = cand[i:j+1][::-1]
            c, _ = self.split(cand)
            if c < best_c: best_c=c; best=cand; improved=True; break
            
    return best

def _update_fitness_heavy(self):
    n = len(self.pop)
    self.pop.sort(key=lambda x: x[0])
    cost_ranks = {id(ind): i for i, ind in enumerate(self.pop)}
    
    dists = np.zeros(n)
    for i in range(n):
        others = random.sample(self.pop, min(n, 5)) 
        d = sum(self.p.dist_chrom(self.pop[i][1], o[1]) for o in others)
        dists[i] = d
    div_ranks = {id(self.pop[idx]): r for r, idx in enumerate(np.argsort(-dists))}
    
    elite_ratio = 0.95 
    fit_list = []
    for ind in self.pop:
        f = cost_ranks[id(ind)] + (1-elite_ratio) * div_ranks[id(ind)]
        fit_list.append((f, ind))
    return fit_list

Problem.split = split_heavy
Problem.educate = educate_heavy
GeneticSolver._update_fitness = _update_fitness_heavy


p = Problem(100, density=1.0, alpha=1, beta=2.0, seed=42)
baseline = p.baseline()

print(f"SCENARIO: Punitivo HEAVY (Beta=2.0)")
print(f"Baseline: {baseline:.2f}")

solver = GeneticSolver(p, pop_size=50) 

final_c, final_r = solver.run(max_generations=1000, time_limit=600)

gap = (baseline - final_c) / baseline * 100
print(f"\nMIGLIORAMENTO FINALE: {gap:.2f}%")

In [ ]:
import math
import random
import time
import networkx as nx
import numpy as np

class Problem:
    def __init__(self, num_cities=50, alpha=1.0, beta=1.0, density=0.5, seed=42):
        self._alpha = alpha
        self._beta = beta
        rng = np.random.default_rng(seed)
        
        cities = rng.random(size=(num_cities, 2))
        cities[0] = [0.5, 0.5] 
        self._graph = nx.Graph()
        self._graph.add_node(0, pos=(cities[0,0], cities[0,1]), gold=0)
        for c in range(1, num_cities):
            self._graph.add_node(c, pos=(cities[c,0], cities[c,1]), gold=(1 + 999 * rng.random()))

        coords = cities
        diff = coords[:, np.newaxis, :] - coords[np.newaxis, :, :]
        dists = np.sqrt(np.sum(diff**2, axis=-1))
        
        for i in range(num_cities):
            for j in range(i + 1, num_cities):
                if rng.random() < density or j == i + 1:
                    self._graph.add_edge(i, j, dist=dists[i, j])
        
        if not nx.is_connected(self._graph):
            comps = list(nx.connected_components(self._graph))
            for i in range(len(comps)-1):
                u, v = list(comps[i])[0], list(comps[i+1])[0]
                self._graph.add_edge(u, v, dist=np.linalg.norm(cities[u]-cities[v]))

        self._sp_lens = dict(nx.all_pairs_dijkstra_path_length(self._graph, weight='dist'))
        self._paths = dict(nx.all_pairs_dijkstra_path(self._graph, weight='dist'))
        self._edge_dists = {}
        for u, v, data in self._graph.edges(data=True):
            self._edge_dists[(u, v)] = self._edge_dists[(v, u)] = data['dist']
        self._golds = [self._graph.nodes[i]['gold'] for i in range(num_cities)]

    @property
    def graph(self): return self._graph
    @property
    def beta(self): return self._beta

    def _get_path_cost(self, u, v, load):
        if u == v: return 0.0
        path = self._paths[u][v]
        cost = 0.0
        for k in range(len(path)-1):
            n1, n2 = path[k], path[k+1]
            d = self._edge_dists.get((n1, n2), 0.0)
            cost += d + (self._alpha * d * load) ** self._beta
        return cost

    def baseline(self):
        total = 0
        for dest in range(1, len(self._graph)):
            path = self._paths[0][dest]
            cost_out = 0; cost_in = 0
            for k in range(len(path)-1):
                d = self._edge_dists.get((path[k], path[k+1]))
                cost_out += d + (self._alpha * d * 0) ** self._beta
            g = self._golds[dest]
            rev_path = path[::-1]
            for k in range(len(rev_path)-1):
                d = self._edge_dists.get((rev_path[k], rev_path[k+1]))
                cost_in += d + (self._alpha * d * g) ** self._beta
            total += cost_out + cost_in
        return total

    def split(self, seq):
        n = len(seq)
        V = [1e30] * (n + 1); V[0] = 0
        P = [0] * (n + 1)
        
        max_tour_len = 40 
        
        for i in range(n):
            if V[i] >= 1e29: continue
            load = 0.0; cost = 0.0; u = 0
            limit = min(n + 1, i + 1 + max_tour_len)
            
            for j in range(i + 1, limit):
                v = seq[j-1]
                try: cost += self._get_path_cost(u, v, load)
                except: break
                load += self._golds[v]
                try: cost_ret = self._get_path_cost(v, 0, load)
                except: break
                
                total = cost + cost_ret
                if V[i] + total < V[j]:
                    V[j] = V[i] + total
                    P[j] = i
                u = v
        
        routes = []; curr = n
        if V[n] >= 1e29: return 1e30, []
        while curr > 0:
            prev = P[curr]
            routes.append([0] + seq[prev:curr] + [0])
            curr = prev
        return V[n], routes[::-1]

    def gen_greedy(self):
        unvisited = set(range(1, len(self._graph)))
        curr = 0; tour = []
        while unvisited:
            best_n = min(unvisited, key=lambda x: self._sp_lens[curr][x])
            tour.append(best_n); unvisited.remove(best_n); curr = best_n
        return tour

    def gen_cw(self):
        n = len(self._graph); nodes = range(1, n)
        savings = []
        d0 = self._sp_lens[0]
        for i in nodes:
            for j in range(i+1, n):
                s = d0[i] + d0[j] - self._sp_lens[i][j]
                if s > 0: savings.append((s, i, j))
        savings.sort(reverse=True, key=lambda x: x[0])
        routes = {i: [i] for i in nodes}
        for _, i, j in savings:
            if routes[i] is not routes[j]:
                r1, r2 = routes[i], routes[j]
                if r1[0] == i and r2[-1] == j: new = r2 + r1
                elif r1[-1] == i and r2[0] == j: new = r1 + r2
                elif r1[0] == i and r2[0] == j: new = r1[::-1] + r2
                elif r1[-1] == i and r2[-1] == j: new = r1 + r2[::-1]
                else: continue
                for x in new: routes[x] = new
        unique = []; seen = set()
        for r in routes.values():
            if id(r) not in seen: unique.extend(r); seen.add(id(r))
        return unique

    def dist_chrom(self, c1, c2):
        e2 = set()
        for i in range(len(c2)-1): e2.add(tuple(sorted((c2[i], c2[i+1]))))
        broken = 0
        for i in range(len(c1)-1):
            if tuple(sorted((c1[i], c1[i+1]))) not in e2: broken += 1
        return broken / max(1, len(c1))

    def educate(self, chrom):
        best = chrom[:]
        best_c, _ = self.split(best)
        improved = True
        attempts = 300 
        
        while improved:
            improved = False
            for _ in range(attempts):
                n = len(best); i = random.randint(0, n-1); j = random.randint(0, n-2)
                cand = best[:i] + best[i+1:]; cand.insert(j, best[i])
                c, _ = self.split(cand)
                if c < best_c: best_c=c; best=cand; improved=True; break
            if improved: continue
            for _ in range(attempts):
                n = len(best); i, j = random.sample(range(n), 2)
                cand = best[:]; cand[i], cand[j] = cand[j], cand[i]
                c, _ = self.split(cand)
                if c < best_c: best_c=c; best=cand; improved=True; break
            if improved: continue
            for _ in range(attempts):
                n = len(best); i, j = sorted(random.sample(range(n), 2))
                cand = best[:]; cand[i:j+1] = cand[i:j+1][::-1]
                c, _ = self.split(cand)
                if c < best_c: best_c=c; best=cand; improved=True; break
        return best

    def ox_crossover(self, p1, p2):
        size = len(p1)
        a, b = sorted(random.sample(range(size), 2))
        child = [-1]*size
        child[a:b+1] = p1[a:b+1]
        p2_vals = [x for x in p2 if x not in set(child[a:b+1])]
        cur = 0
        for i in range(size):
            if child[i] == -1: child[i] = p2_vals[cur]; cur += 1
        return child
    
    def mutate(self, chrom):
        if random.random() < 0.5:
            n = len(chrom); i, j = random.sample(range(n), 2)
            chrom[i], chrom[j] = chrom[j], chrom[i]
        else:
            n = len(chrom); i = random.randint(0, n-4); j = i + 3
            sub = chrom[i:j]; random.shuffle(sub)
            chrom[i:j] = sub
        return chrom

class GeneticSolver:
    def __init__(self, problem, pop_size=50):
        self.p = problem
        self.pop_size = pop_size
        self.pop = [] 
        self.best_sol = None
        self.best_cost = 1e30

    def _init_pop(self):
        seeds = []
        try: seeds.append(self.p.gen_cw())
        except: pass
        try: seeds.append(self.p.gen_greedy())
        except: pass
        
        base = list(range(1, len(self.p.graph)))
        while len(seeds) < self.pop_size:
            r = base[:]; random.shuffle(r)
            seeds.append(r)
        
        self.pop = []
        for s in seeds:
            c, r = self.p.split(s)
            self.pop.append([c, s, r])
            if c < self.best_cost: self.best_cost = c; self.best_sol = r

    def _update_fitness(self):
        n = len(self.pop)
        self.pop.sort(key=lambda x: x[0])
        cost_ranks = {id(ind): i for i, ind in enumerate(self.pop)}
        dists = np.zeros(n)
        for i in range(n):
            others = random.sample(self.pop, min(n, 5)) 
            d = sum(self.p.dist_chrom(self.pop[i][1], o[1]) for o in others)
            dists[i] = d
        div_ranks = {id(self.pop[idx]): r for r, idx in enumerate(np.argsort(-dists))}
        fit_list = []
        elite_ratio = 0.95 
        for ind in self.pop:
            f = cost_ranks[id(ind)] + (1-elite_ratio) * div_ranks[id(ind)]
            fit_list.append((f, ind))
        return fit_list

    def run(self, max_generations=1000, time_limit=300):
        t0 = time.time()
        print(f"--- HGS HEAVY (Pop={self.pop_size}, Gen={max_generations}) ---")
        self._init_pop()
        print(f"   [Init] Best: {self.best_cost:.2f}")

        gen = 0
        while gen < max_generations and (time.time()-t0) < time_limit:
            fits = self._update_fitness()
            p1 = min(random.sample(fits, 2), key=lambda x: x[0])[1]
            p2 = min(random.sample(fits, 2), key=lambda x: x[0])[1]
            
            child_chrom = self.p.ox_crossover(p1[1], p2[1])
            if random.random() < 0.05: child_chrom = self.p.mutate(child_chrom)
            
            child_chrom = self.p.educate(child_chrom)
            child_cost, child_r = self.p.split(child_chrom)
            
            is_clone = False
            if abs(child_cost - self.best_cost) < 1e-3:
                 if self.p.dist_chrom(child_chrom, self.best_sol[0]) == 0: is_clone = True
            
            if not is_clone:
                self.pop.append([child_cost, child_chrom, child_r])
                if child_cost < self.best_cost:
                    gap = (self.p.baseline() - child_cost)/self.p.baseline()*100
                    print(f"   ★ NEW BEST: {child_cost:.2f} (Gap {gap:.2f}%) [Gen {gen}]")
                    self.best_cost = child_cost; self.best_sol = child_r
                
                if len(self.pop) > self.pop_size:
                    fits = self._update_fitness()
                    worst_id = max(fits, key=lambda x: x[0])[1]
                    self.pop.remove(worst_id)
            
            gen += 1
            if gen % 20 == 0: 
                print(f"   Gen {gen:<4} | Best: {self.best_cost:.2f}")

        print(f"--- FINE ---")
        print(f"Final Best: {self.best_cost:.2f}")
        print(f"Total Time: {time.time()-t0:.2f}s")
        return self.best_cost, self.best_sol

p = Problem(100, density=1.0, alpha=1, beta=2.0)
baseline = p.baseline()

print(f"SCENARIO: Punitivo HEAVY (Beta=2.0)")
print(f"Baseline: {baseline:.2f}")

solver = GeneticSolver(p, pop_size=50) 
final_c, final_r = solver.run(max_generations=1000, time_limit=600)

gap = (baseline - final_c) / baseline * 100
print(f"\nMIGLIORAMENTO FINALE: {gap:.2f}%")

SCENARIO: Punitivo HEAVY (Beta=2.0)
Baseline: 5404978.09
--- HGS HEAVY (Pop=50, Gen=1000) ---
   [Init] Best: 5111858.54
   ★ NEW BEST: 4298666.53 (Gap 20.47%) [Gen 0]
   ★ NEW BEST: 4239584.94 (Gap 21.56%) [Gen 1]
   ★ NEW BEST: 4235410.77 (Gap 21.64%) [Gen 6]
   ★ NEW BEST: 4224826.44 (Gap 21.83%) [Gen 7]


In [ ]:
import math
import random
import time
import networkx as nx
import numpy as np

class Problem:
    def __init__(self, num_cities=50, alpha=1.0, beta=1.0, density=0.5, seed=42):
        self._alpha = alpha
        self._beta = beta
        rng = np.random.default_rng(seed)
        
        cities = rng.random(size=(num_cities, 2))
        cities[0] = [0.5, 0.5] 
        self._graph = nx.Graph()
        self._graph.add_node(0, pos=(cities[0,0], cities[0,1]), gold=0)
        for c in range(1, num_cities):
            self._graph.add_node(c, pos=(cities[c,0], cities[c,1]), gold=(1 + 999 * rng.random()))

        coords = cities
        diff = coords[:, np.newaxis, :] - coords[np.newaxis, :, :]
        dists = np.sqrt(np.sum(diff**2, axis=-1))
        
        for i in range(num_cities):
            for j in range(i + 1, num_cities):
                if rng.random() < density or j == i + 1:
                    self._graph.add_edge(i, j, dist=dists[i, j])
        
        if not nx.is_connected(self._graph):
            comps = list(nx.connected_components(self._graph))
            for i in range(len(comps)-1):
                u, v = list(comps[i])[0], list(comps[i+1])[0]
                self._graph.add_edge(u, v, dist=np.linalg.norm(cities[u]-cities[v]))

        self._sp_lens = dict(nx.all_pairs_dijkstra_path_length(self._graph, weight='dist'))
        self._paths = dict(nx.all_pairs_dijkstra_path(self._graph, weight='dist'))
        self._edge_dists = {}
        for u, v, data in self._graph.edges(data=True):
            self._edge_dists[(u, v)] = self._edge_dists[(v, u)] = data['dist']
        self._golds = [self._graph.nodes[i]['gold'] for i in range(num_cities)]

    @property
    def graph(self): return self._graph
    @property
    def beta(self): return self._beta

    def _get_path_cost(self, u, v, load):
        if u == v: return 0.0
        path = self._paths[u][v]
        cost = 0.0
        for k in range(len(path)-1):
            n1, n2 = path[k], path[k+1]
            d = self._edge_dists.get((n1, n2), 0.0)
            cost += d + (self._alpha * d * load) ** self._beta
        return cost

    def baseline(self):
        total = 0
        for dest in range(1, len(self._graph)):
            path = self._paths[0][dest]
            cost_out = 0; cost_in = 0
            for k in range(len(path)-1):
                d = self._edge_dists.get((path[k], path[k+1]))
                cost_out += d + (self._alpha * d * 0) ** self._beta
            g = self._golds[dest]
            rev_path = path[::-1]
            for k in range(len(rev_path)-1):
                d = self._edge_dists.get((rev_path[k], rev_path[k+1]))
                cost_in += d + (self._alpha * d * g) ** self._beta
            total += cost_out + cost_in
        return total

    def split(self, seq):
        n = len(seq)
        V = [1e30] * (n + 1); V[0] = 0
        P = [0] * (n + 1)
        
        max_tour_len = 40 
        
        for i in range(n):
            if V[i] >= 1e29: continue
            load = 0.0; cost = 0.0; u = 0
            limit = min(n + 1, i + 1 + max_tour_len)
            
            for j in range(i + 1, limit):
                v = seq[j-1]
                try: cost += self._get_path_cost(u, v, load)
                except: break
                load += self._golds[v]
                try: cost_ret = self._get_path_cost(v, 0, load)
                except: break
                
                total = cost + cost_ret
                if V[i] + total < V[j]:
                    V[j] = V[i] + total
                    P[j] = i
                u = v
        
        routes = []; curr = n
        if V[n] >= 1e29: return 1e30, []
        while curr > 0:
            prev = P[curr]
            routes.append([0] + seq[prev:curr] + [0])
            curr = prev
        return V[n], routes[::-1]

    def gen_greedy(self):
        unvisited = set(range(1, len(self._graph)))
        curr = 0; tour = []
        while unvisited:
            best_n = min(unvisited, key=lambda x: self._sp_lens[curr][x])
            tour.append(best_n); unvisited.remove(best_n); curr = best_n
        return tour

    def gen_cw(self):
        n = len(self._graph); nodes = range(1, n)
        savings = []
        d0 = self._sp_lens[0]
        for i in nodes:
            for j in range(i+1, n):
                s = d0[i] + d0[j] - self._sp_lens[i][j]
                if s > 0: savings.append((s, i, j))
        savings.sort(reverse=True, key=lambda x: x[0])
        routes = {i: [i] for i in nodes}
        for _, i, j in savings:
            if routes[i] is not routes[j]:
                r1, r2 = routes[i], routes[j]
                if r1[0] == i and r2[-1] == j: new = r2 + r1
                elif r1[-1] == i and r2[0] == j: new = r1 + r2
                elif r1[0] == i and r2[0] == j: new = r1[::-1] + r2
                elif r1[-1] == i and r2[-1] == j: new = r1 + r2[::-1]
                else: continue
                for x in new: routes[x] = new
        unique = []; seen = set()
        for r in routes.values():
            if id(r) not in seen: unique.extend(r); seen.add(id(r))
        return unique

    def dist_chrom(self, c1, c2):
        e2 = set()
        for i in range(len(c2)-1): e2.add(tuple(sorted((c2[i], c2[i+1]))))
        broken = 0
        for i in range(len(c1)-1):
            if tuple(sorted((c1[i], c1[i+1]))) not in e2: broken += 1
        return broken / max(1, len(c1))

    def educate(self, chrom):
        best = chrom[:]
        best_c, _ = self.split(best)
        improved = True
        attempts = 300 
        
        while improved:
            improved = False
            for _ in range(attempts):
                n = len(best); i = random.randint(0, n-1); j = random.randint(0, n-2)
                cand = best[:i] + best[i+1:]; cand.insert(j, best[i])
                c, _ = self.split(cand)
                if c < best_c: best_c=c; best=cand; improved=True; break
            if improved: continue
            for _ in range(attempts):
                n = len(best); i, j = random.sample(range(n), 2)
                cand = best[:]; cand[i], cand[j] = cand[j], cand[i]
                c, _ = self.split(cand)
                if c < best_c: best_c=c; best=cand; improved=True; break
            if improved: continue
            for _ in range(attempts):
                n = len(best); i, j = sorted(random.sample(range(n), 2))
                cand = best[:]; cand[i:j+1] = cand[i:j+1][::-1]
                c, _ = self.split(cand)
                if c < best_c: best_c=c; best=cand; improved=True; break
        return best

    def ox_crossover(self, p1, p2):
        size = len(p1)
        a, b = sorted(random.sample(range(size), 2))
        child = [-1]*size
        child[a:b+1] = p1[a:b+1]
        p2_vals = [x for x in p2 if x not in set(child[a:b+1])]
        cur = 0
        for i in range(size):
            if child[i] == -1: child[i] = p2_vals[cur]; cur += 1
        return child
    
    def mutate(self, chrom):
        if random.random() < 0.5:
            n = len(chrom); i, j = random.sample(range(n), 2)
            chrom[i], chrom[j] = chrom[j], chrom[i]
        else:
            n = len(chrom); i = random.randint(0, n-4); j = i + 3
            sub = chrom[i:j]; random.shuffle(sub)
            chrom[i:j] = sub
        return chrom

class GeneticSolver:
    def __init__(self, problem, pop_size=50):
        self.p = problem
        self.pop_size = pop_size
        self.pop = [] 
        self.best_sol = None
        self.best_cost = 1e30

    def _init_pop(self):
        seeds = []
        try: seeds.append(self.p.gen_cw())
        except: pass
        try: seeds.append(self.p.gen_greedy())
        except: pass
        
        base = list(range(1, len(self.p.graph)))
        while len(seeds) < self.pop_size:
            r = base[:]; random.shuffle(r)
            seeds.append(r)
        
        self.pop = []
        for s in seeds:
            c, r = self.p.split(s)
            self.pop.append([c, s, r])
            if c < self.best_cost: self.best_cost = c; self.best_sol = r

    def _update_fitness(self):
        n = len(self.pop)
        self.pop.sort(key=lambda x: x[0])
        cost_ranks = {id(ind): i for i, ind in enumerate(self.pop)}
        dists = np.zeros(n)
        for i in range(n):
            others = random.sample(self.pop, min(n, 5)) 
            d = sum(self.p.dist_chrom(self.pop[i][1], o[1]) for o in others)
            dists[i] = d
        div_ranks = {id(self.pop[idx]): r for r, idx in enumerate(np.argsort(-dists))}
        fit_list = []
        elite_ratio = 0.95 
        for ind in self.pop:
            f = cost_ranks[id(ind)] + (1-elite_ratio) * div_ranks[id(ind)]
            fit_list.append((f, ind))
        return fit_list

    def run(self, max_generations=1000, time_limit=300):
        t0 = time.time()
        print(f"--- HGS SMART (Pop={self.pop_size}, Gen={max_generations}) ---")
        self._init_pop()
        print(f"   [Init] Best: {self.best_cost:.2f}")

        gen = 0
        attempts_light = 30
        attempts_heavy = 300
        
        while gen < max_generations and (time.time()-t0) < time_limit:
            fits = self._update_fitness()
            p1 = min(random.sample(fits, 2), key=lambda x: x[0])[1]
            p2 = min(random.sample(fits, 2), key=lambda x: x[0])[1]
            
            child_chrom = self.p.ox_crossover(p1[1], p2[1])
            if random.random() < 0.1: child_chrom = self.p.mutate(child_chrom)
            
            raw_c, _ = self.p.split(child_chrom)
            
            gap_from_best = (raw_c - self.best_cost) / self.best_cost
            
            if gap_from_best < 0.2: 
                child_chrom = self.p.educate(child_chrom) 
            else:
                pass 

            child_cost, child_r = self.p.split(child_chrom)
            
            is_clone = False
            if abs(child_cost - self.best_cost) < 1e-3:
                 if self.p.dist_chrom(child_chrom, self.best_sol[0]) == 0: is_clone = True
            
            if not is_clone:
                self.pop.append([child_cost, child_chrom, child_r])
                if child_cost < self.best_cost:
                    gap = (self.p.baseline() - child_cost)/self.p.baseline()*100
                    print(f"   ★ NEW BEST: {child_cost:.2f} (Gap {gap:.2f}%) [Gen {gen}]")
                    self.best_cost = child_cost; self.best_sol = child_r
                
                if len(self.pop) > self.pop_size:
                    fits = self._update_fitness()
                    worst_id = max(fits, key=lambda x: x[0])[1]
                    self.pop.remove(worst_id)
            
            gen += 1
            if gen % 50 == 0: 
                print(f"   Gen {gen:<4} | Best: {self.best_cost:.2f}")

        print(f"--- FINE ---")
        print(f"Final Best: {self.best_cost:.2f}")
        print(f"Total Time: {time.time()-t0:.2f}s")
        return self.best_cost, self.best_sol

p = Problem(100, density=1.0, alpha=1, beta=2.0, seed=42)
baseline = p.baseline()

print(f"SCENARIO: Punitivo HEAVY (Beta=2.0)")
print(f"Baseline: {baseline:.2f}")

solver = GeneticSolver(p, pop_size=50) 
final_c, final_r = solver.run(max_generations=1000, time_limit=600)

gap = (baseline - final_c) / baseline * 100
print(f"\nMIGLIORAMENTO FINALE: {gap:.2f}%")

SCENARIO: Punitivo HEAVY (Beta=2.0)
Baseline: 5404978.09
--- HGS HEAVY (Pop=50, Gen=1000) ---
   [Init] Best: 5060028.44
   ★ NEW BEST: 4267765.25 (Gap 21.04%) [Gen 0]


In [ ]:
import math
import random
import time
import networkx as nx
import numpy as np
from numba import njit

@njit(fastmath=True)
def get_cost_numba(dist, load, alpha, beta):
    return dist + (alpha * dist * load)**beta

@njit(fastmath=True)
def split_numba(seq, dist_matrix, golds, alpha, beta, max_len=40):
    """
    Versione compilata e ultra-veloce dello Split.
    Ritorna: (Costo Totale, Array dei Predecessori)
    """
    n = len(seq)
    V = np.full(n + 1, 1e30, dtype=np.float64)
    P = np.zeros(n + 1, dtype=np.int32)
    V[0] = 0.0
    
    for i in range(n):
        if V[i] >= 1e29: continue
        
        current_load = 0.0
        current_cost = 0.0
        u = 0 
        
        limit = min(n, i + max_len)
        
        for j in range(i + 1, limit + 1):
            v_idx = seq[j-1] 
            
            d_uv = dist_matrix[u, v_idx]
            current_cost += d_uv + (alpha * d_uv * current_load)**beta
            
            current_load += golds[v_idx]
            
            d_v0 = dist_matrix[v_idx, 0]
            return_cost = d_v0 + (alpha * d_v0 * current_load)**beta
            
            total_trip_cost = current_cost + return_cost
            
            if V[i] + total_trip_cost < V[j]:
                V[j] = V[i] + total_trip_cost
                P[j] = i
                
            u = v_idx

    return V[n], P

@njit(fastmath=True)
def educate_numba(chrom, dist_matrix, golds, alpha, beta, attempts=500):
    """
    Local Search compilata. Fa migliaia di tentativi in millisecondi.
    """
    best_chrom = chrom.copy()
    best_cost, _ = split_numba(best_chrom, dist_matrix, golds, alpha, beta)
    
    n = len(chrom)
    improved = True
    
    while improved:
        improved = False
        
        for _ in range(attempts):
            i = np.random.randint(0, n)
            j = np.random.randint(0, n-1) 
            
            val = best_chrom[i]
            temp = np.empty(n-1, dtype=np.int32)
            idx = 0
            for k in range(n):
                if k != i:
                    temp[idx] = best_chrom[k]
                    idx += 1
            
            cand = np.empty(n, dtype=np.int32)
            cand[:j] = temp[:j]
            cand[j] = val
            cand[j+1:] = temp[j:]
            
            c, _ = split_numba(cand, dist_matrix, golds, alpha, beta)
            if c < best_cost:
                best_cost = c
                best_chrom = cand
                improved = True
                break 
        if improved: continue

        for _ in range(attempts):
            i = np.random.randint(0, n)
            j = np.random.randint(0, n)
            if i == j: continue
            
            cand = best_chrom.copy()
            cand[i], cand[j] = cand[j], cand[i]
            
            c, _ = split_numba(cand, dist_matrix, golds, alpha, beta)
            if c < best_cost:
                best_cost = c
                best_chrom = cand
                improved = True
                break
        if improved: continue
        
        for _ in range(attempts):
            i = np.random.randint(0, n)
            j = np.random.randint(0, n)
            if i > j: i, j = j, i
            if i == j: continue
            
            cand = best_chrom.copy()
            cand[i:j+1] = best_chrom[i:j+1][::-1]
            
            c, _ = split_numba(cand, dist_matrix, golds, alpha, beta)
            if c < best_cost:
                best_cost = c
                best_chrom = cand
                improved = True
                break
                
    return best_chrom

class ProblemNumba:
    def __init__(self, num_cities=50, alpha=1.0, beta=1.0, density=0.5, seed=42):
        self._alpha = float(alpha)
        self._beta = float(beta)
        rng = np.random.default_rng(seed)
        
        cities = rng.random(size=(num_cities, 2))
        cities[0] = [0.5, 0.5]
        
        G = nx.Graph()
        G.add_node(0, pos=cities[0])
        for c in range(1, num_cities):
            G.add_node(c, pos=cities[c])
            
        coords = cities
        diff = coords[:, np.newaxis, :] - coords[np.newaxis, :, :]
        eucl_dists = np.sqrt(np.sum(diff**2, axis=-1))
        
        for i in range(num_cities):
            for j in range(i + 1, num_cities):
                if rng.random() < density or j == i + 1:
                    G.add_edge(i, j, weight=eucl_dists[i, j])
                    
        if not nx.is_connected(G):
             G.add_edge(0, num_cities-1, weight=eucl_dists[0, num_cities-1])

        raw_paths = dict(nx.all_pairs_dijkstra_path_length(G, weight='weight'))
        self.dist_matrix = np.zeros((num_cities, num_cities), dtype=np.float64)
        for i in range(num_cities):
            for j in range(num_cities):
                self.dist_matrix[i, j] = raw_paths[i].get(j, 1e9) 
        
        self.golds = np.zeros(num_cities, dtype=np.float64)
        for c in range(1, num_cities):
            self.golds[c] = (1 + 999 * rng.random())

    def baseline(self):
        chrom = np.arange(1, len(self.golds), dtype=np.int32)
        total = 0.0
        n = len(self.golds)
        for i in range(1, n):
            d_out = self.dist_matrix[0, i]
            d_in = self.dist_matrix[i, 0]
            g = self.golds[i]
            cost = (d_out + (self._alpha * d_out * 0)**self._beta) + \
                   (d_in + (self._alpha * d_in * g)**self._beta)
            total += cost
        return total

    def split_wrapper(self, chrom_list):
        chrom_arr = np.array(chrom_list, dtype=np.int32)
        c, p = split_numba(chrom_arr, self.dist_matrix, self.golds, self._alpha, self._beta)
        
        routes = []
        curr = len(chrom_arr)
        while curr > 0:
            prev = p[curr]
            segment = [0] + chrom_list[prev:curr] + [0]
            routes.append(segment)
            curr = prev
        return c, routes[::-1]

    def educate_wrapper(self, chrom_list):
        chrom_arr = np.array(chrom_list, dtype=np.int32)
        improved_arr = educate_numba(chrom_arr, self.dist_matrix, self.golds, self._alpha, self._beta, attempts=1000)
        return improved_arr.tolist()

    def gen_cw(self):
        n = len(self.golds)
        savings = []
        d = self.dist_matrix
        for i in range(1, n):
            for j in range(i+1, n):
                sav = d[0,i] + d[0,j] - d[i,j]
                if sav > 0: savings.append((sav, i, j))
        savings.sort(reverse=True, key=lambda x: x[0])
        
        routes = {i: [i] for i in range(1, n)}
        for _, i, j in savings:
            if routes[i] is not routes[j]:
                r1 = routes[i]; r2 = routes[j]
                if r1[0] == i and r2[-1] == j: new = r2 + r1
                elif r1[-1] == i and r2[0] == j: new = r1 + r2
                elif r1[0] == i and r2[0] == j: new = r1[::-1] + r2
                elif r1[-1] == i and r2[-1] == j: new = r1 + r2[::-1]
                else: continue
                for x in new: routes[x] = new
        
        unique = []
        seen = set()
        for r in routes.values():
            if id(r) not in seen: unique.extend(r); seen.add(id(r))
        return unique

class GeneticSolverNumba:
    def __init__(self, problem, pop_size=100):
        self.p = problem
        self.pop_size = pop_size
        self.pop = [] 
        self.best_sol = None
        self.best_cost = 1e30

    def _dist(self, c1, c2):
        return 1.0 if c1 != c2 else 0.0

    def _init_pop(self):
        seeds = [self.p.gen_cw()]
        base = list(range(1, len(self.p.golds)))
        while len(seeds) < self.pop_size:
            r = base[:]; random.shuffle(r)
            seeds.append(r)
        
        self.pop = []
        for s in seeds:
            c, r = self.p.split_wrapper(s)
            self.pop.append([c, s, r])
            if c < self.best_cost: self.best_cost = c; self.best_sol = r

    def run(self, max_generations=5000, time_limit=60):
        t0 = time.time()
        print(f"--- HGS NUMBA (Pop={self.pop_size}) ---")
        self._init_pop()
        print(f"   [Init] Best: {self.best_cost:.2f}")

        gen = 0
        dummy = np.arange(1, 10, dtype=np.int32)
        
        while gen < max_generations and (time.time()-t0) < time_limit:
            self.pop.sort(key=lambda x: x[0])
            pool = self.pop[:int(self.pop_size*0.5)]
            if len(pool) < 2: pool = self.pop
            
            p1 = random.choice(pool)[1]
            p2 = random.choice(pool)[1]

            size = len(p1)
            a, b = sorted(random.sample(range(size), 2))
            child = [-1]*size
            child[a:b+1] = p1[a:b+1]
            p2_vals = [x for x in p2 if x not in set(child[a:b+1])]
            cur = 0
            for i in range(size):
                if child[i] == -1: child[i] = p2_vals[cur]; cur += 1
            
            if random.random() < 0.1:
                i, j = random.sample(range(size), 2)
                child[i], child[j] = child[j], child[i]

            child = self.p.educate_wrapper(child)
            
            cost, routes = self.p.split_wrapper(child)
            
            if cost < self.pop[-1][0]: 
                 if abs(cost - self.best_cost) > 1e-4:
                     self.pop.pop()
                     self.pop.append([cost, child, routes])
                     self.pop.sort(key=lambda x: x[0])
            
            if cost < self.best_cost:
                gap = (self.p.baseline() - cost)/self.p.baseline()*100
                print(f"NEW BEST: {cost:.2f} (Gap {gap:.2f}%) [Gen {gen}]")
                self.best_cost = cost; self.best_sol = routes
                
            gen += 1
            if gen % 1000 == 0:
                print(f"   Gen {gen:<5} | Best: {self.best_cost:.2f} | Time: {time.time()-t0:.1f}s")

        print(f"--- FINE ---")
        print(f"Final Best: {self.best_cost:.2f}")
        print(f"Total Time: {time.time()-t0:.2f}s")
        return self.best_cost, self.best_sol

p_numba = ProblemNumba(100, density=1.0, alpha=1, beta=2.0)
baseline = p_numba.baseline()

print(f"SCENARIO: Punitivo (Beta=2.0) - NUMBA ACCELERATED")
print(f"Baseline: {baseline:.2f}")

solver = GeneticSolverNumba(p_numba, pop_size=100)
final_c, final_r = solver.run(max_generations=10000, time_limit=60)

gap = (baseline - final_c) / baseline * 100
print(f"\nMIGLIORAMENTO FINALE: {gap:.2f}%")

SCENARIO: Punitivo (Beta=2.0) - NUMBA ACCELERATED
Baseline: 3992425.13
--- 🚀 HGS NUMBA (Pop=100) ---
   [Init] Best: 3773692.82
   ★ NEW BEST: 2945462.57 (Gap 26.22%) [Gen 0]
   ★ NEW BEST: 2945421.27 (Gap 26.22%) [Gen 3]
   ★ NEW BEST: 2936116.81 (Gap 26.46%) [Gen 17]
   ★ NEW BEST: 2934738.07 (Gap 26.49%) [Gen 24]
--- FINE ---
Final Best: 2934738.07
Total Time: 61.36s

MIGLIORAMENTO FINALE: 26.49%


In [ ]:
import time
import random
import networkx as nx
import numpy as np

class ProblemPython:
    def __init__(self, num_cities=50, alpha=1.0, beta=1.0, density=0.5, seed=42):
        self._alpha = alpha
        self._beta = beta
        self.seed = seed
        rng = np.random.default_rng(seed)
        
        cities = rng.random(size=(num_cities, 2))
        cities[0] = [0.5, 0.5] 
        self._graph = nx.Graph()
        self._graph.add_node(0, pos=(cities[0,0], cities[0,1]), gold=0)
        for c in range(1, num_cities):
            self._graph.add_node(c, pos=(cities[c,0], cities[c,1]), gold=(1 + 999 * rng.random()))

        coords = cities
        diff = coords[:, np.newaxis, :] - coords[np.newaxis, :, :]
        dists = np.sqrt(np.sum(diff**2, axis=-1))
        for i in range(num_cities):
            for j in range(i + 1, num_cities):
                if rng.random() < density or j == i + 1:
                    self._graph.add_edge(i, j, dist=dists[i, j])
        if not nx.is_connected(self._graph):
            comps = list(nx.connected_components(self._graph))
            for i in range(len(comps)-1):
                u, v = list(comps[i])[0], list(comps[i+1])[0]
                self._graph.add_edge(u, v, dist=np.linalg.norm(cities[u]-cities[v]))

        self._sp_lens = dict(nx.all_pairs_dijkstra_path_length(self._graph, weight='dist'))
        self._paths = dict(nx.all_pairs_dijkstra_path(self._graph, weight='dist'))
        self._edge_dists = {}
        for u, v, data in self._graph.edges(data=True):
            self._edge_dists[(u, v)] = self._edge_dists[(v, u)] = data['dist']
        self._golds = [self._graph.nodes[i]['gold'] for i in range(num_cities)]

    @property
    def graph(self): return self._graph
    
    @property
    def beta(self): return self._beta

    def baseline(self):
        total = 0
        for dest in range(1, len(self._graph)):
            path = self._paths[0][dest]
            cost_out = 0; cost_in = 0
            for k in range(len(path)-1):
                d = self._edge_dists.get((path[k], path[k+1]))
                cost_out += d + (self._alpha * d * 0) ** self._beta
            g = self._golds[dest]
            rev_path = path[::-1]
            for k in range(len(rev_path)-1):
                d = self._edge_dists.get((rev_path[k], rev_path[k+1]))
                cost_in += d + (self._alpha * d * g) ** self._beta
            total += cost_out + cost_in
        return total

    def _get_path_cost(self, u, v, load):
        if u == v: return 0.0
        path = self._paths[u][v]
        cost = 0.0
        for k in range(len(path)-1):
            n1, n2 = path[k], path[k+1]
            d = self._edge_dists.get((n1, n2), 0.0)
            cost += d + (self._alpha * d * load) ** self._beta
        return cost

    def split(self, seq):
        n = len(seq)
        V = [1e30] * (n + 1); V[0] = 0
        P = [0] * (n + 1)
        max_tour_len = 40 
        for i in range(n):
            if V[i] >= 1e29: continue
            load = 0.0; cost = 0.0; u = 0
            limit = min(n + 1, i + 1 + max_tour_len)
            for j in range(i + 1, limit):
                v = seq[j-1]
                try: cost += self._get_path_cost(u, v, load)
                except: break
                load += self._golds[v]
                try: cost_ret = self._get_path_cost(v, 0, load)
                except: break
                total = cost + cost_ret
                if V[i] + total < V[j]:
                    V[j] = V[i] + total; P[j] = i
                u = v
        routes = []; curr = n
        if V[n] >= 1e29: return 1e30, []
        while curr > 0:
            prev = P[curr]; routes.append([0] + seq[prev:curr] + [0]); curr = prev
        return V[n], routes[::-1]

    def educate(self, chrom):
        best = chrom[:]; best_c, _ = self.split(best)
        improved = True; attempts = 150
        while improved:
            improved = False
            for _ in range(attempts): 
                n = len(best); i = random.randint(0, n-1); j = random.randint(0, n-2)
                cand = best[:i] + best[i+1:]; cand.insert(j, best[i])
                c, _ = self.split(cand)
                if c < best_c: best_c=c; best=cand; improved=True; break
            if improved: continue
            for _ in range(attempts): 
                n = len(best); i, j = random.sample(range(n), 2)
                cand = best[:]; cand[i], cand[j] = cand[j], cand[i]
                c, _ = self.split(cand)
                if c < best_c: best_c=c; best=cand; improved=True; break
            if improved: continue
            for _ in range(attempts):
                n = len(best); i, j = sorted(random.sample(range(n), 2))
                cand = best[:]; cand[i:j+1] = cand[i:j+1][::-1]
                c, _ = self.split(cand)
                if c < best_c: best_c=c; best=cand; improved=True; break
        return best

    def ox_crossover(self, p1, p2):
        size = len(p1); a, b = sorted(random.sample(range(size), 2))
        child = [-1]*size; child[a:b+1] = p1[a:b+1]
        p2_vals = [x for x in p2 if x not in set(child[a:b+1])]
        cur = 0
        for i in range(size):
            if child[i] == -1: child[i] = p2_vals[cur]; cur += 1
        return child
    
    def mutate(self, chrom):
        if random.random() < 0.5:
            n = len(chrom); i, j = random.sample(range(n), 2)
            chrom[i], chrom[j] = chrom[j], chrom[i]
        else:
            n = len(chrom); i = random.randint(0, n-4); j = i + 3
            sub = chrom[i:j]; random.shuffle(sub)
            chrom[i:j] = sub
        return chrom
    
    def gen_greedy(self):
        unvisited = set(range(1, len(self._graph))); curr = 0; tour = []
        while unvisited:
            best_n = min(unvisited, key=lambda x: self._sp_lens[curr][x])
            tour.append(best_n); unvisited.remove(best_n); curr = best_n
        return tour

class GeneticSolverPython:
    def __init__(self, problem, pop_size=50):
        self.p = problem
        self.pop_size = pop_size
        self.pop = []; self.best_cost = 1e30

    def run(self, max_generations=1000, time_limit=300):
        t0 = time.time()
        print(f"--- HGS PYTHON (Pop={self.pop_size}, Gen={max_generations}) ---")
        
        seeds = [self.p.gen_greedy()]
        base = list(range(1, len(self.p.graph))) 
        while len(seeds) < self.pop_size:
            r = base[:]; random.shuffle(r); seeds.append(r)
        
        self.pop = []
        for s in seeds:
            c, r = self.p.split(s)
            self.pop.append([c, s, r])
            if c < self.best_cost: self.best_cost = c
            
        gen = 0
        while gen < max_generations and (time.time()-t0) < time_limit:
            self.pop.sort(key=lambda x: x[0])
            idx1 = random.randint(0, min(len(self.pop)-1, self.pop_size//2))
            idx2 = random.randint(0, min(len(self.pop)-1, self.pop_size//2))
            p1 = self.pop[idx1][1]
            p2 = self.pop[idx2][1]
            
            child = self.p.ox_crossover(p1, p2)
            if random.random() < 0.1: child = self.p.mutate(child)
            
            child = self.p.educate(child)
            cost, routes = self.p.split(child)
            
            if cost < self.pop[-1][0]:
                 if abs(cost - self.best_cost) > 1e-4:
                     self.pop.pop(); self.pop.append([cost, child, routes])
            
            if cost < self.best_cost:
                gap = (self.p.baseline() - cost)/self.p.baseline()*100
                print(f"   ★ NEW BEST: {cost:.2f} (Gap {gap:.2f}%) [Gen {gen}]")
                self.best_cost = cost
            gen += 1
            if gen % 100 == 0:
                print(f"   Gen {gen:<4} | Best: {self.best_cost:.2f}")
            
        t_end = time.time()-t0
        print(f"   DONE. Best: {self.best_cost:.2f} | Time: {t_end:.2f}s | Speed: {gen/max(1,t_end):.1f} gen/s")
        return self.best_cost, t_end

SEED = 42
N_CITIES = 100
BETA = 2.0 
DENSITY = 1.0 

print(f"BENCHMARK: PYTHON vs NUMBA")
print(f"Scenario: {N_CITIES} Città, Beta={BETA}, Seed={SEED}")
print("="*60)

p_py = ProblemPython(N_CITIES, density=DENSITY, alpha=1, beta=BETA, seed=SEED)
base_py = p_py.baseline()
print(f"Baseline (Python): {base_py:.2f}")

solver_py = GeneticSolverPython(p_py, pop_size=50)
cost_py, time_py = solver_py.run(max_generations=5000, time_limit=60) 

print("-" * 60)

try:
    p_nb = ProblemNumba(N_CITIES, density=DENSITY, alpha=1, beta=BETA, seed=SEED)
    base_nb = p_nb.baseline()
    print(f"Baseline (Numba):  {base_nb:.2f} (Check Identicità: {abs(base_nb-base_py)<0.1})")

    solver_nb = GeneticSolverNumba(p_nb, pop_size=100)
    cost_nb, time_nb = solver_nb.run(max_generations=10000, time_limit=60)

    print("="*60)
    print(f"RISULTATI FINALI (Dopo 60s ciascuno):")
    print(f"PYTHON Pure: Gap {(base_py-cost_py)/base_py*100:.2f}%")
    print(f"NUMBA JIT:   Gap {(base_nb-cost_nb)/base_nb*100:.2f}%")
    
    if cost_nb < cost_py:
        print(f"VINCITORE: NUMBA")
    else:
        print(f"VINCITORE: PYTHON")

except NameError:
    print("ERRORE: Esegui prima la cella con il codice NUMBA (GeneticSolverNumba)!")

🏆 BENCHMARK: PYTHON vs NUMBA
Scenario: 100 Città, Beta=2.0, Seed=42
Baseline (Python): 5404978.09
--- 🐢 HGS PYTHON (Pop=50, Gen=5000) ---
   ★ NEW BEST: 4354982.94 (Gap 19.43%) [Gen 0]
   ★ NEW BEST: 4295415.63 (Gap 20.53%) [Gen 2]
   ★ NEW BEST: 4291657.78 (Gap 20.60%) [Gen 3]
   DONE. Best: 4291657.78 | Time: 70.88s | Speed: 0.1 gen/s
------------------------------------------------------------
Baseline (Numba):  3992425.13 (Check Identicità: False)
--- 🚀 HGS NUMBA (Pop=100) ---
   [Init] Best: 3746002.46
   ★ NEW BEST: 2938978.03 (Gap 26.39%) [Gen 0]
   ★ NEW BEST: 2925862.12 (Gap 26.71%) [Gen 8]
--- FINE ---
Final Best: 2925862.12
Total Time: 60.25s
RISULTATI FINALI (Dopo 60s ciascuno):
PYTHON Pure: Gap 20.60%
NUMBA JIT:   Gap 26.71%
VINCITORE: NUMBA 🏆


In [ ]:
import math
import random
import time
import networkx as nx
import numpy as np
from numba import njit


@njit(fastmath=True)
def get_cost_numba(dist, load, alpha, beta):
    return dist + (alpha * dist * load)**beta

@njit(fastmath=True)
def split_numba(seq, dist_matrix, golds, alpha, beta, max_len=50):

    n = len(seq)
    V = np.full(n + 1, 1e30, dtype=np.float64)
    P = np.zeros(n + 1, dtype=np.int32)
    V[0] = 0.0
    
    for i in range(n):
        if V[i] >= 1e29: continue
        
        current_load = 0.0
        current_cost = 0.0
        u = 0 
        limit = min(n, i + max_len)
        
        for j in range(i + 1, limit + 1):
            v_idx = seq[j-1]
            
            d_uv = dist_matrix[u, v_idx]
            current_cost += d_uv + (alpha * d_uv * current_load)**beta
            
            current_load += golds[v_idx]
            
            d_v0 = dist_matrix[v_idx, 0]
            return_cost = d_v0 + (alpha * d_v0 * current_load)**beta
            
            total_trip_cost = current_cost + return_cost
            
            if V[i] + total_trip_cost < V[j]:
                V[j] = V[i] + total_trip_cost
                P[j] = i
                
            u = v_idx

    return V[n], P

@njit(fastmath=True)
def educate_numba(chrom, dist_matrix, golds, alpha, beta, attempts=1000):
    best_chrom = chrom.copy()
    best_cost, _ = split_numba(best_chrom, dist_matrix, golds, alpha, beta)
    n = len(chrom)
    improved = True
    
    while improved:
        improved = False
        
        for _ in range(attempts):
            i = np.random.randint(0, n)
            j = np.random.randint(0, n-1)
            
            val = best_chrom[i]
            temp = np.empty(n, dtype=np.int32)
            idx = 0
            for k in range(n):
                if k != i:
                    temp[idx] = best_chrom[k]
                    idx += 1
            
            cand = np.empty(n, dtype=np.int32)
            cand[:j] = temp[:j]
            cand[j] = val
            cand[j+1:] = temp[j:n-1]
            
            c, _ = split_numba(cand, dist_matrix, golds, alpha, beta)
            if c < best_cost:
                best_cost = c; best_chrom = cand; improved = True; break
        if improved: continue

        for _ in range(attempts):
            i = np.random.randint(0, n)
            j = np.random.randint(0, n)
            if i == j: continue
            
            cand = best_chrom.copy()
            cand[i], cand[j] = cand[j], cand[i]
            
            c, _ = split_numba(cand, dist_matrix, golds, alpha, beta)
            if c < best_cost:
                best_cost = c; best_chrom = cand; improved = True; break
        if improved: continue
        
        for _ in range(attempts):
            i = np.random.randint(0, n)
            j = np.random.randint(0, n)
            if i > j: i, j = j, i
            if i == j: continue
            
            cand = best_chrom.copy()
            cand[i:j+1] = best_chrom[i:j+1][::-1]
            
            c, _ = split_numba(cand, dist_matrix, golds, alpha, beta)
            if c < best_cost:
                best_cost = c; best_chrom = cand; improved = True; break
                
    return best_chrom

class ProblemNumba:
    def __init__(self, num_cities=50, alpha=1.0, beta=1.0, density=0.5, seed=42):
        self._alpha = float(alpha)
        self._beta = float(beta)
        rng = np.random.default_rng(seed)
        
        cities = rng.random(size=(num_cities, 2))
        cities[0] = [0.5, 0.5]
        
        G = nx.Graph()
        G.add_node(0, pos=cities[0])
        for c in range(1, num_cities):
            G.add_node(c, pos=cities[c])
            
        coords = cities
        diff = coords[:, np.newaxis, :] - coords[np.newaxis, :, :]
        eucl_dists = np.sqrt(np.sum(diff**2, axis=-1))
        
        for i in range(num_cities):
            for j in range(i + 1, num_cities):
                if rng.random() < density or j == i + 1:
                    G.add_edge(i, j, weight=eucl_dists[i, j])
                    
        if not nx.is_connected(G):
             G.add_edge(0, num_cities-1, weight=eucl_dists[0, num_cities-1])

        raw_paths = dict(nx.all_pairs_dijkstra_path_length(G, weight='weight'))
        self.dist_matrix = np.zeros((num_cities, num_cities), dtype=np.float64)
        for i in range(num_cities):
            for j in range(num_cities):
                self.dist_matrix[i, j] = raw_paths[i].get(j, 1e9) 
        
        self.golds = np.zeros(num_cities, dtype=np.float64)
        for c in range(1, num_cities):
            self.golds[c] = (1 + 999 * rng.random())

    def baseline(self):
        total = 0.0
        n = len(self.golds)
        for i in range(1, n):
            d_out = self.dist_matrix[0, i]
            d_in = self.dist_matrix[i, 0]
            g = self.golds[i]
            cost = (d_out + (self._alpha * d_out * 0)**self._beta) + \
                   (d_in + (self._alpha * d_in * g)**self._beta)
            total += cost
        return total

    def split_wrapper(self, chrom_list):
        chrom_arr = np.array(chrom_list, dtype=np.int32)
        c, p = split_numba(chrom_arr, self.dist_matrix, self.golds, self._alpha, self._beta)
        routes = []
        curr = len(chrom_arr)
        while curr > 0:
            prev = p[curr]
            segment = [0] + chrom_list[prev:curr] + [0]
            routes.append(segment)
            curr = prev
        return c, routes[::-1]

    def educate_wrapper(self, chrom_list):
        chrom_arr = np.array(chrom_list, dtype=np.int32)
        improved_arr = educate_numba(chrom_arr, self.dist_matrix, self.golds, self._alpha, self._beta, attempts=1000)
        return improved_arr.tolist()

    def gen_cw(self):
        n = len(self.golds); savings = []; d = self.dist_matrix
        for i in range(1, n):
            for j in range(i+1, n):
                sav = d[0,i] + d[0,j] - d[i,j]
                if sav > 0: savings.append((sav, i, j))
        savings.sort(reverse=True, key=lambda x: x[0])
        routes = {i: [i] for i in range(1, n)}
        for _, i, j in savings:
            if routes[i] is not routes[j]:
                r1 = routes[i]; r2 = routes[j]
                if r1[0] == i and r2[-1] == j: new = r2 + r1
                elif r1[-1] == i and r2[0] == j: new = r1 + r2
                elif r1[0] == i and r2[0] == j: new = r1[::-1] + r2
                elif r1[-1] == i and r2[-1] == j: new = r1 + r2[::-1]
                else: continue
                for x in new: routes[x] = new
        unique = []; seen = set()
        for r in routes.values():
            if id(r) not in seen: unique.extend(r); seen.add(id(r))
        return unique

class GeneticSolverNumba:
    def __init__(self, problem, pop_size=100):
        self.p = problem
        self.pop_size = pop_size
        self.pop = [] 
        self.best_sol = None
        self.best_cost = 1e30

    def _init_pop(self):
        seeds = [self.p.gen_cw()]
        base = list(range(1, len(self.p.golds)))
        while len(seeds) < self.pop_size:
            r = base[:]; random.shuffle(r)
            seeds.append(r)
        
        self.pop = []
        for s in seeds:
            c, r = self.p.split_wrapper(s)
            self.pop.append([c, s, r])
            if c < self.best_cost: self.best_cost = c; self.best_sol = r

    def run(self, max_generations=10000, time_limit=60):
        t0 = time.time()
        print(f"--- HGS NUMBA (Pop={self.pop_size}) ---")
        self._init_pop()
        print(f"   [Init] Best: {self.best_cost:.2f}")

        gen = 0
        dummy = np.arange(1, 10, dtype=np.int32)
        
        while gen < max_generations and (time.time()-t0) < time_limit:
            self.pop.sort(key=lambda x: x[0])
            pool = self.pop[:int(self.pop_size*0.6)] 
            p1 = random.choice(pool)[1]
            p2 = random.choice(pool)[1]
            
            size = len(p1)
            a, b = sorted(random.sample(range(size), 2))
            child = [-1]*size
            child[a:b+1] = p1[a:b+1]
            p2_vals = [x for x in p2 if x not in set(child[a:b+1])]
            cur = 0
            for i in range(size):
                if child[i] == -1: child[i] = p2_vals[cur]; cur += 1
            
            if random.random() < 0.1:
                i, j = random.sample(range(size), 2)
                child[i], child[j] = child[j], child[i]

            child = self.p.educate_wrapper(child)
            cost, routes = self.p.split_wrapper(child)
            
            if cost < self.pop[-1][0]:
                 if abs(cost - self.best_cost) > 1e-4: 
                     self.pop.pop()
                     self.pop.append([cost, child, routes])
                     self.pop.sort(key=lambda x: x[0])
            
            if cost < self.best_cost:
                gap = (self.p.baseline() - cost)/self.p.baseline()*100
                print(f"   ★ NEW BEST: {cost:.2f} (Gap {gap:.2f}%) [Gen {gen}]")
                self.best_cost = cost; self.best_sol = routes
                
            gen += 1
            if gen % 1000 == 0:
                print(f"   Gen {gen:<5} | Best: {self.best_cost:.2f} | Time: {time.time()-t0:.1f}s")

        print(f"--- FINE ---")
        print(f"Final Best: {self.best_cost:.2f}")
        print(f"Total Time: {time.time()-t0:.2f}s")
        return self.best_cost, self.best_sol

p_numba = ProblemNumba(100, density=1.0, alpha=1, beta=2.0, seed=42)
baseline = p_numba.baseline()

print(f"SCENARIO: Punitivo (Beta=2.0) - NUMBA VERSION")
print(f"Baseline: {baseline:.2f}")

solver = GeneticSolverNumba(p_numba, pop_size=100)
final_c, final_r = solver.run(max_generations=10000, time_limit=60)

gap = (baseline - final_c) / baseline * 100
print(f"\nMIGLIORAMENTO FINALE: {gap:.2f}%")

SCENARIO: Punitivo (Beta=2.0) - NUMBA VERSION
Baseline: 3992425.13
--- ☢️  HGS NUMBA NUCLEAR (Pop=100) ---
   [Init] Best: 3753032.50
   ★ NEW BEST: 3017270.34 (Gap 24.43%) [Gen 0]
   ★ NEW BEST: 2963384.54 (Gap 25.77%) [Gen 1]
   ★ NEW BEST: 2958249.92 (Gap 25.90%) [Gen 2]
   ★ NEW BEST: 2957540.52 (Gap 25.92%) [Gen 3]
   ★ NEW BEST: 2939071.46 (Gap 26.38%) [Gen 7]
   ★ NEW BEST: 2935286.19 (Gap 26.48%) [Gen 12]
   ★ NEW BEST: 2932249.91 (Gap 26.55%) [Gen 21]
--- FINE ---
Final Best: 2932249.91
Total Time: 61.95s

MIGLIORAMENTO FINALE: 26.55%


In [ ]:
import math
import random
import time
import networkx as nx
import numpy as np
from numba import njit

@njit(fastmath=True)
def split_numba(seq, dist_matrix, golds, alpha, beta, max_len=50):
    n = len(seq)
    V = np.full(n + 1, 1e30, dtype=np.float64)
    P = np.zeros(n + 1, dtype=np.int32)
    V[0] = 0.0
    
    for i in range(n):
        if V[i] >= 1e29: continue
        current_load = 0.0; current_cost = 0.0; u = 0
        limit = min(n, i + max_len)
        for j in range(i + 1, limit + 1):
            v_idx = seq[j-1]
            d_uv = dist_matrix[u, v_idx]
            current_cost += d_uv + (alpha * d_uv * current_load)**beta
            current_load += golds[v_idx]
            d_v0 = dist_matrix[v_idx, 0]
            return_cost = d_v0 + (alpha * d_v0 * current_load)**beta
            total_trip_cost = current_cost + return_cost
            if V[i] + total_trip_cost < V[j]:
                V[j] = V[i] + total_trip_cost; P[j] = i
            u = v_idx
    return V[n], P

@njit(fastmath=True)
def educate_numba(chrom, dist_matrix, golds, alpha, beta, attempts=500):
    best_chrom = chrom.copy()
    best_cost, _ = split_numba(best_chrom, dist_matrix, golds, alpha, beta)
    n = len(chrom); improved = True
    while improved:
        improved = False
        for _ in range(attempts):
            i = np.random.randint(0, n); j = np.random.randint(0, n-1)
            val = best_chrom[i]
            temp = np.empty(n, dtype=np.int32); idx=0
            for k in range(n):
                if k!=i: temp[idx]=best_chrom[k]; idx+=1
            cand = np.empty(n, dtype=np.int32)
            cand[:j] = temp[:j]; cand[j] = val; cand[j+1:] = temp[j:n-1]
            c, _ = split_numba(cand, dist_matrix, golds, alpha, beta)
            if c < best_cost: best_cost=c; best_chrom=cand; improved=True; break
        if improved: continue
        for _ in range(attempts):
            i = np.random.randint(0, n); j = np.random.randint(0, n)
            if i==j: continue
            cand = best_chrom.copy(); cand[i], cand[j] = cand[j], cand[i]
            c, _ = split_numba(cand, dist_matrix, golds, alpha, beta)
            if c < best_cost: best_cost=c; best_chrom=cand; improved=True; break
    return best_chrom

class ProblemInstance:
    def __init__(self, num_cities=50, alpha=1.0, beta=1.0, density=0.5, seed=42):
        self._alpha = float(alpha)
        self._beta = float(beta)
        rng = np.random.default_rng(seed)
        cities = rng.random(size=(num_cities, 2)); cities[0] = [0.5, 0.5]
        G = nx.Graph(); G.add_node(0, pos=cities[0])
        for c in range(1, num_cities): G.add_node(c, pos=cities[c])
        coords = cities; diff = coords[:, np.newaxis, :] - coords[np.newaxis, :, :]
        eucl_dists = np.sqrt(np.sum(diff**2, axis=-1))
        for i in range(num_cities):
            for j in range(i + 1, num_cities):
                if rng.random() < density or j == i + 1: G.add_edge(i, j, weight=eucl_dists[i, j])
        if not nx.is_connected(G): G.add_edge(0, num_cities-1, weight=eucl_dists[0, num_cities-1])
        raw_paths = dict(nx.all_pairs_dijkstra_path_length(G, weight='weight'))
        self.dist_matrix = np.zeros((num_cities, num_cities), dtype=np.float64)
        for i in range(num_cities):
            for j in range(num_cities): self.dist_matrix[i, j] = raw_paths[i].get(j, 1e9)
        self.golds = np.zeros(num_cities, dtype=np.float64)
        for c in range(1, num_cities): self.golds[c] = (1 + 999 * rng.random())

    def baseline(self):
        total = 0.0; n = len(self.golds)
        for i in range(1, n):
            d_out = self.dist_matrix[0, i]; d_in = self.dist_matrix[i, 0]; g = self.golds[i]
            cost = (d_out + (self._alpha * d_out * 0)**self._beta) + (d_in + (self._alpha * d_in * g)**self._beta)
            total += cost
        return total
    
    def evaluate(self, chrom):
        arr = np.array(chrom, dtype=np.int32)
        c, _ = split_numba(arr, self.dist_matrix, self.golds, self._alpha, self._beta)
        return c

class HGS_Solver:
    def __init__(self, problem, pop_size=100):
        self.p = problem; self.pop_size = pop_size; self.best_cost = 1e30
    def run(self, max_gen=2000, time_limit=60): 
        t0 = time.time()
        print(f"--- [HGS] STARTING ---")
        pop = []; base = list(range(1, len(self.p.golds)))
        for _ in range(self.pop_size):
            r = base[:]; random.shuffle(r); pop.append(r)
        
        for i in range(len(pop)):
            pop[i] = self.p.evaluate(pop[i]), pop[i]
            if pop[i][0] < self.best_cost: self.best_cost = pop[i][0]

        gen = 0
        while gen < max_gen and (time.time()-t0) < time_limit:
            pop.sort(key=lambda x: x[0])
            p1 = random.choice(pop[:int(self.pop_size*0.5)])[1]
            p2 = random.choice(pop[:int(self.pop_size*0.5)])[1]
            
            size = len(p1); a, b = sorted(random.sample(range(size), 2))
            child = [-1]*size; child[a:b+1] = p1[a:b+1]
            p2_vals = [x for x in p2 if x not in set(child[a:b+1])]
            cur = 0
            for i in range(size):
                if child[i] == -1: child[i] = p2_vals[cur]; cur += 1
            
            child_arr = np.array(child, dtype=np.int32)
            child_arr = educate_numba(child_arr, self.p.dist_matrix, self.p.golds, self.p._alpha, self.p._beta, 300)
            child = child_arr.tolist()
            
            cost = self.p.evaluate(child)
            
            if cost < self.best_cost:
                self.best_cost = cost
      
            if cost < pop[-1][0]:
                pop.pop(); pop.append((cost, child))
            
            gen += 1
        return self.best_cost

def init_population(problem, size, dist_matrix):
    pop = []; base = list(range(1, len(problem.golds)))
    for _ in range(size):
        r = base[:]; random.shuffle(r); pop.append(r)
    return pop

def getFastSmartCost(problem, chrom, dist_matrix):
    return problem.evaluate(chrom)

def tournament_selection(problem, pop, dm, k):
    sample = random.sample(pop, k)
    return min(sample, key=lambda x: getFastSmartCost(problem, x, dm))

def crossover_ox(p1, p2):
    size = len(p1); a, b = sorted(random.sample(range(size), 2))
    child = [-1]*size; child[a:b+1] = p1[a:b+1]
    p2_vals = [x for x in p2 if x not in set(child[a:b+1])]
    cur = 0
    for i in range(size):
        if child[i] == -1: child[i] = p2_vals[cur]; cur += 1
    return child

def mutate(chrom, rate):
    if random.random() < rate:
        i, j = random.sample(range(len(chrom)), 2)
        chrom[i], chrom[j] = chrom[j], chrom[i]
    return chrom

def genetic_algorithm_user(problem, dist_matrix, max_time=60):
    POPULATION_SIZE = 100
    NUM_GENERATIONS = 1000000 
    TOURNAMENT_SIZE = 5
    MUTATION_RATE = 0.1
    
    t0 = time.time()
    print("--- [BASIC GA] STARTING ---")
    
    population = init_population(problem, POPULATION_SIZE, dist_matrix)
    best_solution = min(population, key=lambda x: getFastSmartCost(problem, x, dist_matrix))
    best_fitness = getFastSmartCost(problem, best_solution, dist_matrix)
    
    stagnation_counter = 0

    for gen in range(NUM_GENERATIONS):
        if (time.time() - t0) > max_time: break
        
        new_population = []
        
        # Elitismo
        population.sort(key=lambda x: getFastSmartCost(problem, x, dist_matrix))
        new_population.append(population[0])
        
        current_best_fitness = getFastSmartCost(problem, population[0], dist_matrix)
        if current_best_fitness < best_fitness:
            best_fitness = current_best_fitness
            best_solution = population[0]
            stagnation_counter = 0
        else:
            stagnation_counter += 1

        current_mutation_rate = MUTATION_RATE
        if stagnation_counter > 50:
            current_mutation_rate = 0.5 
        
        while len(new_population) < POPULATION_SIZE:
            p1 = population[random.randint(0, 50)]
            p2 = population[random.randint(0, 50)]
            
            child = crossover_ox(p1, p2)
            child = mutate(child, current_mutation_rate)
            new_population.append(child)
            
        population = new_population
    
    return best_fitness

SEED = 42
N_CITIES = 200
BETA = 5.0
DENSITY = 0.2
TIME_LIMIT = 60 

print(f"SHOWDOWN: HGS vs BASIC GA")
print(f"Scenario: {N_CITIES} Città, Beta={BETA}, Seed={SEED}")
print(f"Tempo concesso: {TIME_LIMIT}s per algoritmo")
print("="*60)

problem = ProblemInstance(N_CITIES, alpha=1, beta=BETA, density=DENSITY, seed=SEED)
baseline = problem.baseline()
print(f"Baseline (A/R): {baseline:.2f}")
print("-" * 60)

hgs_solver = HGS_Solver(problem, pop_size=100)
cost_hgs = hgs_solver.run(max_gen=100000, time_limit=TIME_LIMIT)
gap_hgs = (baseline - cost_hgs)/baseline*100
print(f"HGS Result:      {cost_hgs:.2f} (Gap {gap_hgs:.2f}%)")

print("-" * 60)

cost_basic = genetic_algorithm_user(problem, problem.dist_matrix, max_time=TIME_LIMIT)
gap_basic = (baseline - cost_basic)/baseline*100
print(f"Basic GA Result: {cost_basic:.2f} (Gap {gap_basic:.2f}%)")

print("="*60)
if cost_hgs < cost_basic:
    print(f"WINNER: HGS (Better by {cost_basic - cost_hgs:.2f})")
else:
    print(f"WINNER: BASIC GA (Better by {cost_hgs - cost_basic:.2f})")

🥊 SHOWDOWN: HGS vs BASIC GA
Scenario: 200 Città, Beta=5.0, Seed=42
Tempo concesso: 60s per algoritmo
Baseline (A/R): 1119546571563957.12
------------------------------------------------------------
--- [HGS] STARTING ---
HGS Result:      492182790224514.75 (Gap 56.04%)
------------------------------------------------------------
--- [BASIC GA] STARTING ---
Basic GA Result: 477975927436292.62 (Gap 57.31%)
🏆 WINNER: BASIC GA (Better by 14206862788222.12)


In [ ]:
import math
import random
import time
import networkx as nx
import numpy as np
from numba import njit

@njit(fastmath=True)
def split_numba_ultra(seq, dist_matrix, golds, alpha, beta, max_len=60):
    n = len(seq)
    V = np.full(n + 1, 1e30, dtype=np.float64)
    P = np.zeros(n + 1, dtype=np.int32)
    V[0] = 0.0
    for i in range(n):
        if V[i] >= 1e29: continue
        current_load = 0.0; current_cost = 0.0; u = 0
        limit = min(n, i + max_len)
        for j in range(i + 1, limit + 1):
            v_idx = seq[j-1]
            d_uv = dist_matrix[u, v_idx]
            current_cost += d_uv + (alpha * d_uv * current_load)**beta
            current_load += golds[v_idx]
            d_v0 = dist_matrix[v_idx, 0]
            return_cost = d_v0 + (alpha * d_v0 * current_load)**beta
            total_trip_cost = current_cost + return_cost
            if V[i] + total_trip_cost < V[j]:
                V[j] = V[i] + total_trip_cost; P[j] = i
            u = v_idx
    return V[n], P

@njit(fastmath=True)
def educate_numba_ultra(chrom, dist_matrix, golds, alpha, beta, attempts=2000):
    best_chrom = chrom.copy()
    best_cost, _ = split_numba_ultra(best_chrom, dist_matrix, golds, alpha, beta)
    n = len(chrom)
    
    for _ in range(attempts):
        mode = np.random.randint(0, 3)
        i = np.random.randint(0, n)
        j = np.random.randint(0, n)
        if i == j: continue
        
        cand = best_chrom.copy()
        if mode == 0: 
            cand[i], cand[j] = cand[j], cand[i]
        elif mode == 1:
            if i > j: i, j = j, i
            cand[i:j+1] = best_chrom[i:j+1][::-1]
        else: 
            val = best_chrom[i]
            if i < j:
                cand[i:j] = best_chrom[i+1:j+1]
                cand[j] = val
            else:
                cand[j+1:i+1] = best_chrom[j:i]
                cand[j] = val
        
        c, _ = split_numba_ultra(cand, dist_matrix, golds, alpha, beta)
        if c < best_cost:
            best_cost = c
            best_chrom = cand
    return best_chrom

class ProblemArena:
    def __init__(self, n=100, b=2.0, seed=42):
        self.alpha, self.beta = 1.0, float(b)
        rng = np.random.default_rng(seed)
        cities = rng.random((n, 2)); cities[0] = [0.5, 0.5]
        self.golds = np.zeros(n); self.golds[1:] = 1 + 999 * rng.random(n-1)
        diff = cities[:, None, :] - cities[None, :, :]
        self.dist_matrix = np.sqrt(np.sum(diff**2, axis=-1))

    def evaluate(self, chrom):
        c, _ = split_numba_ultra(np.array(chrom, dtype=np.int32), self.dist_matrix, self.golds, self.alpha, self.beta)
        return c

def hgs_nuclear(prob, time_limit=60):
    t0 = time.time()
    n = len(prob.golds)
    pop_size = 50
    pop = []
    base = list(range(1, n))
    for _ in range(pop_size):
        c = base[:]; random.shuffle(c)
        pop.append([prob.evaluate(c), c])
    
    best_c = min([x[0] for x in pop])
    while (time.time() - t0) < time_limit:
        pop.sort(key=lambda x: x[0])
        p1, p2 = pop[0][1], pop[1][1]
        
        # OX Crossover
        size = len(p1); a, b = sorted(random.sample(range(size), 2))
        child = [-1]*size; child[a:b+1] = p1[a:b+1]
        p2_vals = [x for x in p2 if x not in set(child[a:b+1])]
        cur = 0
        for i in range(size):
            if child[i] == -1: child[i] = p2_vals[cur]; cur += 1
        
        child_arr = educate_numba_ultra(np.array(child, dtype=np.int32), prob.dist_matrix, prob.golds, prob.alpha, prob.beta)
        child_cost = prob.evaluate(child_arr)
        
        if child_cost < pop[-1][0]:
            pop[-1] = [child_cost, child_arr.tolist()]
        
        if child_cost < best_c:
            best_c = child_cost
    return best_c

def basic_ga_user(prob, time_limit=60):
    t0 = time.time()
    n = len(prob.golds)
    pop_size = 50
    pop = []
    base = list(range(1, n))
    for _ in range(pop_size):
        c = base[:]; random.shuffle(c)
        pop.append([prob.evaluate(c), c])
    
    best_c = min([x[0] for x in pop])
    stagnation = 0
    while (time.time() - t0) < time_limit:
        pop.sort(key=lambda x: x[0])
        curr_best = pop[0][0]
        if curr_best < best_c:
            best_c = curr_best; stagnation = 0
        else: stagnation += 1
        
        mut_rate = 0.1 if stagnation < 50 else 0.5
        new_pop = [pop[0]]
        while len(new_pop) < pop_size:
            p1, p2 = random.sample(pop[:25], 2)
            size = len(p1[1]); a, b = sorted(random.sample(range(size), 2))
            c = [-1]*size; c[a:b+1] = p1[1][a:b+1]
            p2_v = [x for x in p2[1] if x not in set(c[a:b+1])]
            cur = 0
            for i in range(size):
                if c[i] == -1: c[i] = p2_v[cur]; cur += 1
            if random.random() < mut_rate:
                idx1, idx2 = random.sample(range(size), 2)
                c[idx1], c[idx2] = c[idx2], c[idx1]
            new_pop.append([prob.evaluate(c), c])
        pop = new_pop
    return best_c

print("PREPARAZIONE: HGS NUCLEAR vs BASIC GA")
prob = ProblemArena(n=200, b=3.0, seed=42)
TIME = 60

print(f"\n--- RUNNING BASIC GA ({TIME}s) ---")
res_ga = basic_ga_user(prob, TIME)
print(f"Risultato Basic GA: {res_ga:.2f}")

print(f"\n--- RUNNING HGS NUCLEAR ({TIME}s) ---")
res_hgs = hgs_nuclear(prob, TIME)
print(f"Risultato HGS Nuclear: {res_hgs:.2f}")

print("\n" + "="*40)
print(f"DISTACCO: {res_ga - res_hgs:.2f} punti")
print(f"MIGLIORAMENTO HGS: {((res_ga - res_hgs)/res_ga*100):.2f}%")
print("="*40)

🚀 PREPARAZIONE ARENA: HGS NUCLEAR vs BASIC GA

--- RUNNING BASIC GA (60s) ---
Risultato Basic GA: 1902262703.87

--- RUNNING HGS NUCLEAR (60s) ---
Risultato HGS Nuclear: 1873978334.26

DISTACCO: 28284369.61 punti
MIGLIORAMENTO HGS: 1.49%


In [ ]:
import math
import random
import time
import networkx as nx
import numpy as np
from numba import njit

@njit(fastmath=True)
def split_numba_final(seq, dist_matrix, golds, alpha, beta, max_len=60):
    n = len(seq)
    V = np.full(n + 1, 1e30, dtype=np.float64)
    P = np.zeros(n + 1, dtype=np.int32)
    V[0] = 0.0
    for i in range(n):
        if V[i] >= 1e29: continue
        current_load = 0.0; current_cost = 0.0; u = 0
        limit = min(n, i + max_len)
        for j in range(i + 1, limit + 1):
            v_idx = seq[j-1]
            d_uv = dist_matrix[u, v_idx]
            current_cost += d_uv + (alpha * d_uv * current_load)**beta
            current_load += golds[v_idx]
            d_v0 = dist_matrix[v_idx, 0]
            return_cost = d_v0 + (alpha * d_v0 * current_load)**beta
            total_trip_cost = current_cost + return_cost
            if V[i] + total_trip_cost < V[j]:
                V[j] = V[i] + total_trip_cost; P[j] = i
            u = v_idx
    return V[n], P

@njit(fastmath=True)
def educate_numba_fast(chrom, dist_matrix, golds, alpha, beta, attempts=50):
    best_chrom = chrom.copy()
    best_cost, _ = split_numba_final(best_chrom, dist_matrix, golds, alpha, beta)
    n = len(chrom)
    
    for _ in range(attempts):
        mode = np.random.randint(0, 3)
        i, j = np.random.randint(0, n), np.random.randint(0, n)
        if i == j: continue
        
        cand = best_chrom.copy()
        if mode == 0: # Swap
            cand[i], cand[j] = cand[j], cand[i]
        elif mode == 1: # 2-Opt
            if i > j: i, j = j, i
            cand[i:j+1] = best_chrom[i:j+1][::-1]
        else: # Relocate
            val = best_chrom[i]
            if i < j:
                cand[i:j] = best_chrom[i+1:j+1]
                cand[j] = val
            else:
                cand[j+1:i+1] = best_chrom[j:i]
                cand[j] = val
        
        c, _ = split_numba_final(cand, dist_matrix, golds, alpha, beta)
        if c < best_cost:
            best_cost = c; best_chrom = cand
    return best_chrom

def hgs_fast_exploration(prob, time_limit=60):
    t0 = time.time()
    n = len(prob.golds)
    pop_size = 200 
    pop = []
    base = list(range(1, n))
    for _ in range(pop_size):
        c = base[:]; random.shuffle(c)
        pop.append([prob.evaluate(c), c])
    
    best_c = min([x[0] for x in pop])
    while (time.time() - t0) < time_limit:
        pop.sort(key=lambda x: x[0])
        p1 = pop[random.randint(0, 20)][1]
        p2 = pop[random.randint(0, 50)][1]
        
        size = len(p1); a, b = sorted(random.sample(range(size), 2))
        child = [-1]*size; child[a:b+1] = p1[a:b+1]
        p2_vals = [x for x in p2 if x not in set(child[a:b+1])]
        cur = 0
        for i in range(size):
            if child[i] == -1: child[i] = p2_vals[cur]; cur += 1
        
        child_arr = educate_numba_fast(np.array(child, dtype=np.int32), prob.dist_matrix, prob.golds, prob.alpha, prob.beta)
        child_cost = prob.evaluate(child_arr)
        
        if child_cost < pop[-1][0]:
            pop[-1] = [child_cost, child_arr.tolist()]
        
        if child_cost < best_c:
            best_c = child_cost
    return best_c

print("ROUND FINALE: HGS EXPLORER vs BASIC GA")
prob = ProblemArena(n=200, b=5.0, seed=42)
TIME = 60

print(f"\n--- RUNNING BASIC GA ({TIME}s) ---")
res_ga = basic_ga_user(prob, TIME) 
print(f"Risultato Basic GA: {res_ga:.2e}")

print(f"\n--- RUNNING HGS EXPLORER ({TIME}s) ---")
res_hgs = hgs_fast_exploration(prob, TIME)
print(f"Risultato HGS Explorer: {res_hgs:.2e}")

print("\n" + "="*40)
diff = res_ga - res_hgs
if diff > 0:
    print(f"VINCITORE: HGS (Sotto di {diff:.2e})")
else:
    print(f"VINCITORE: BASIC GA (Sotto di {-diff:.2e})")
print("="*40)

🥊 ROUND FINALE: HGS EXPLORER vs BASIC GA

--- RUNNING BASIC GA (60s) ---
Risultato Basic GA: 1.10e+14

--- RUNNING HGS EXPLORER (60s) ---
Risultato HGS Explorer: 1.33e+14

VINCITORE: BASIC GA 🐢 (Sotto di 2.32e+13)
